In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# load the data
print("Loading dataset...")
df = pd.read_csv('/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings.csv')

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Years covered: {df['Year'].min()} - {df['Year'].max()}")
print(f"Unique companies: {df['gvkey'].nunique()}")

print(list(df.columns))

In [ ]:
import pandas as pd
import numpy as np

def fix_annualized_returns_in_factor_dataset(df):
    """
    apply additional filtering to annualized returns in the factor loadings dataset
    to remove extreme values that snuck back in through annualization
    """
    
    print("Fixing annualized returns in factor loadings dataset")
    print(f"Input dataset: {df.shape}")
    
    # identify annualized return columns
    annualized_cols = [col for col in df.columns if 'annualized' in col]
    raw_return_cols = [col for col in df.columns if 'return_' in col and 'excess' not in col and 'annualized' not in col]
    excess_return_cols = [col for col in df.columns if 'excess_return_' in col and 'annualized' not in col]
    
    print(f"Found columns:")
    print(f"   Raw returns (non-annualized): {len(raw_return_cols)}")
    print(f"   Excess returns (non-annualized): {len(excess_return_cols)}")  
    print(f"   Annualized returns: {len(annualized_cols)}")
    
    # check current statistics before fixing
    print(f"Before fixing - extreme value counts:")
    for col in annualized_cols:
        if col in df.columns:
            values = df[col].dropna()
            if len(values) > 0:
                extreme_count = (values.abs() > 2.0).sum()  # >200%
                very_extreme = (values.abs() > 5.0).sum()   # >500%
                
                print(f"   {col[:30]:30}: {extreme_count:4d} >200% ({extreme_count/len(values):.1%}), "
                      f"{very_extreme:4d} >500% ({very_extreme/len(values):.1%})")
    
    # apply reasonable bounds to annualized returns
    print("Applying quality filters to annualized returns")
    print("Bounds: -95% to +500% (reasonable for annualized individual stock returns)")
    
    fixed_count = 0
    total_observations = 0
    
    for col in annualized_cols:
        if col in df.columns:
            original_data = df[col].copy()
            valid_data = original_data.dropna()
            
            if len(valid_data) > 0:
                # apply reasonable bounds for annualized returns
                lower_bound = -0.95  # -95% (total loss)
                upper_bound = 5.00   # +500% (very high but possible for individual stocks)
                
                # count what will be filtered
                too_low = (valid_data < lower_bound).sum()
                too_high = (valid_data > upper_bound).sum()
                total_filtered = too_low + too_high
                
                # apply filter
                df[col] = df[col].where(
                    (df[col] >= lower_bound) & (df[col] <= upper_bound), 
                    np.nan
                )
                
                # report results
                remaining = df[col].notna().sum()
                print(f"   {col[:30]:30}: filtered {total_filtered:4d}/{len(valid_data):4d} "
                      f"({total_filtered/len(valid_data):5.1%}), remaining: {remaining:4d}")
                
                fixed_count += total_filtered
                total_observations += len(valid_data)
    
    print(f"Filtering summary:")
    print(f"   Total annualized observations processed: {total_observations:,}")
    print(f"   Total extreme values filtered: {fixed_count:,}")
    print(f"   Filtering rate: {fixed_count/total_observations:.1%}")
    
    return df

# run the fix
print("Return filtering fix for factor loadings dataset")

# paths
enhanced_dataset_path = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_enhanced_dataset.csv"
factor_loadings_path = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings.csv"

print("What this fixes:")
print("Keeps the excellent factor loadings (40% r squared)")
print("Keeps clean non-annualized returns") 
print("Filters extreme annualized returns that snuck back in")
print("Makes dataset consistent with the original filtering")

# clean the data
clean_df = fix_annualized_returns_in_factor_dataset(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats.mstats import winsorize

def create_regression_variables_summary(df, winsorize_level=0.01):
    """
    create comprehensive summary of regression variables with winsorization analysis
    """
    
    print("regression variables analysis & winsorization")
    
    # define key regression variables
    regression_vars = {
        # dependent variables (y)
        'Dependent Variables (Y)': {
            'excess_return_3mo': '3-Month Excess Return',
            'excess_return_6mo': '6-Month Excess Return', 
            'excess_return_9mo': '9-Month Excess Return',
            'excess_return_12mo': '12-Month Excess Return'
        },
        
        # control variables (x)
        'Control Variables': {
            'calc_log_market_cap': 'Log Market Cap (Size)',
            'calc_price_to_book': 'Price-to-Book (Value)',
            'calc_roa': 'ROA (Profitability)'
        },
        
        # factor betas (3-month window as example)
        'Factor Betas (3mo)': {
            'beta_mktrf_t3': 'Market Beta (MKTRF)',
            'beta_smb_t3': 'Size Beta (SMB)',
            'beta_hml_t3': 'Value Beta (HML)',
            'beta_rmw_t3': 'Profitability Beta (RMW)',
            'beta_cma_t3': 'Investment Beta (CMA)', 
            'beta_umd_t3': 'Momentum Beta (UMD)'
        }
    }
    
    # flatten variable dictionary for processing
    all_vars = {}
    for category, vars_dict in regression_vars.items():
        all_vars.update(vars_dict)
    
    # filter to only existing columns
    existing_vars = {var: desc for var, desc in all_vars.items() if var in df.columns}
    print(f"found {len(existing_vars)} regression variables in dataset")
    
    # create summary statistics before winsorization
    print("summary statistics - before winsorization")
    
    summary_before = create_summary_table(df, existing_vars, "Before Winsorization")
    print(summary_before.round(4))
    
    # apply winsorization
    print(f"applying winsorization ({winsorize_level*100:.0f}%/{100-winsorize_level*100:.0f}%)")
    
    df_winsorized = df.copy()
    winsorization_report = {}
    
    for var in existing_vars.keys():
        if var in df.columns:
            original_data = df[var].dropna()
            
            if len(original_data) > 0:
                # apply winsorization
                winsorized_data = winsorize(original_data, limits=[winsorize_level, winsorize_level])
                df_winsorized[var] = df[var].copy()
                df_winsorized.loc[original_data.index, var] = winsorized_data
                
                # track changes
                n_changed = (original_data != winsorized_data).sum()
                pct_changed = (n_changed / len(original_data)) * 100
                
                winsorization_report[var] = {
                    'n_obs': len(original_data),
                    'n_changed': n_changed,
                    'pct_changed': pct_changed,
                    'original_min': original_data.min(),
                    'original_max': original_data.max(),
                    'winsor_min': winsorized_data.min(),
                    'winsor_max': winsorized_data.max()
                }
    
    # display winsorization impact
    print("winsorization impact summary:")
    print(f"{'Variable':30} {'N Obs':>8} {'Changed':>8} {'% Changed':>10} {'Orig Range':>20} {'Winsor Range':>20}")
    print("-" * 105)
    
    for var, report in winsorization_report.items():
        var_name = existing_vars[var][:25]
        orig_range = f"[{report['original_min']:.3f}, {report['original_max']:.3f}]"
        winsor_range = f"[{report['winsor_min']:.3f}, {report['winsor_max']:.3f}]"
        
        print(f"{var_name:30} {report['n_obs']:>8,} {report['n_changed']:>8} {report['pct_changed']:>9.1f}% {orig_range:>20} {winsor_range:>20}")
    
    # create summary statistics after winsorization
    print("summary statistics - after winsorization")
    
    summary_after = create_summary_table(df_winsorized, existing_vars, "After Winsorization")
    print(summary_after.round(4))
    
    return df_winsorized, summary_before, summary_after

def create_summary_table(df, variables_dict, title):
    """create detailed summary statistics table"""
    
    summary_stats = []
    
    for var, description in variables_dict.items():
        if var in df.columns:
            data = df[var].dropna()
            
            if len(data) > 0:
                stats_dict = {
                    'Variable': description,
                    'N': len(data),
                    'Mean': data.mean(),
                    'Std': data.std(),
                    'Min': data.min(),
                    'P25': data.quantile(0.25),
                    'Median': data.median(),
                    'P75': data.quantile(0.75),
                    'Max': data.max(),
                    'Skewness': data.skew(),
                    'Kurtosis': data.kurtosis()
                }
                summary_stats.append(stats_dict)
    
    return pd.DataFrame(summary_stats).set_index('Variable')

# run the complete analysis
df_winsorized, before_stats, after_stats = create_regression_variables_summary(df, winsorize_level=0.01)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def investigate_data_quality_issues(df):
    """
    investigate problematic p/b ratios and factor betas
    """
    
    print("data quality investigation")
    
    # price-to-book investigation
    print("1. price-to-book ratio investigation")
    
    pb_col = 'calc_price_to_book'
    if pb_col in df.columns:
        pb_data = df[pb_col].dropna()
        
        print(f"price-to-book analysis:")
        print(f"   total observations: {len(pb_data):,}")
        print(f"   negative values: {(pb_data < 0).sum():,} ({(pb_data < 0).sum()/len(pb_data)*100:.1f}%)")
        print(f"   extreme negative (<-100): {(pb_data < -100).sum():,}")
        print(f"   extreme positive (>100): {(pb_data > 100).sum():,}")
        print(f"   normal range (0-20): {((pb_data >= 0) & (pb_data <= 20)).sum():,}")
        
        # show most extreme values
        print("most extreme p/b values:")
        print("most negative:")
        extreme_neg = pb_data.nsmallest(5)
        for idx, val in extreme_neg.items():
            company = df.loc[idx, 'Company Name'] if 'Company Name' in df.columns else 'unknown'
            ticker = df.loc[idx, 'Ticker'] if 'Ticker' in df.columns else 'unknown'
            year = df.loc[idx, 'Year'] if 'Year' in df.columns else 'unknown'
            print(f"   {val:8.1f} - {ticker} ({company[:30]}) - {year}")
        
        print("most positive:")
        extreme_pos = pb_data.nlargest(5)
        for idx, val in extreme_pos.items():
            company = df.loc[idx, 'Company Name'] if 'Company Name' in df.columns else 'unknown'
            ticker = df.loc[idx, 'Ticker'] if 'Ticker' in df.columns else 'unknown'
            year = df.loc[idx, 'Year'] if 'Year' in df.columns else 'unknown'
            print(f"   {val:8.1f} - {ticker} ({company[:30]}) - {year}")
    
    # factor betas investigation  
    print("2. factor betas investigation")
    
    beta_cols = [col for col in df.columns if col.startswith('beta_') and col.endswith('_t3')]
    
    for beta_col in beta_cols:
        if beta_col in df.columns:
            beta_data = df[beta_col].dropna()
            factor_name = beta_col.replace('beta_', '').replace('_t3', '').upper()
            
            print(f"{factor_name} beta analysis:")
            print(f"   total observations: {len(beta_data):,}")
            print(f"   extreme negative (<-5): {(beta_data < -5).sum():,}")
            print(f"   extreme positive (>5): {(beta_data > 5).sum():,}")
            print(f"   normal range (-3 to 3): {((beta_data >= -3) & (beta_data <= 3)).sum():,} ({((beta_data >= -3) & (beta_data <= 3)).sum()/len(beta_data)*100:.1f}%)")
            
            # show most extreme betas
            if (beta_data.abs() > 5).any():
                print(f"   extreme {factor_name} betas:")
                extreme_betas = beta_data[beta_data.abs() > 5].sort_values(key=abs, ascending=False).head(3)
                for idx, val in extreme_betas.items():
                    company = df.loc[idx, 'Company Name'] if 'Company Name' in df.columns else 'unknown'
                    ticker = df.loc[idx, 'Ticker'] if 'Ticker' in df.columns else 'unknown'
                    year = df.loc[idx, 'Year'] if 'Year' in df.columns else 'unknown'
                    print(f"      {val:6.2f} - {ticker} ({company[:25]}) - {year}")
    
    # return data quality check
    print("3. return data quality check")
    
    return_cols = ['excess_return_3mo', 'excess_return_6mo', 'excess_return_9mo', 'excess_return_12mo']
    
    for ret_col in return_cols:
        if ret_col in df.columns:
            ret_data = df[ret_col].dropna()
            horizon = ret_col.replace('excess_return_', '').replace('mo', '')
            
            extreme_positive = (ret_data > 1).sum()  # >100%
            extreme_negative = (ret_data < -0.8).sum()  # <-80%
            
            print(f"   {horizon}-month: {extreme_positive} extreme positive, {extreme_negative} extreme negative")
    
    return df

run_data_quality_investigation(df)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import seaborn as sns

def enhanced_winsorization_strategy(df):
    """
    Apply differentiated winsorization strategy based on variable characteristics
    """
    
    print("Enhanced winsorization strategy")
    
    # Define variable-specific winsorization levels
    winsorization_strategy = {
        # More aggressive for highly problematic variables
        'Aggressive (5%/95%)': {
            'variables': ['calc_price_to_book'],
            'level': 0.05,
            'reason': 'High concentration of extreme outliers'
        },
        
        # Moderate for factor betas (small % of outliers)
        'Moderate (2%/98%)': {
            'variables': [
                'beta_mktrf_t3', 'beta_smb_t3', 'beta_hml_t3', 
                'beta_rmw_t3', 'beta_cma_t3', 'beta_umd_t3',
                'beta_mktrf_t6', 'beta_smb_t6', 'beta_hml_t6',
                'beta_rmw_t6', 'beta_cma_t6', 'beta_umd_t6',
                'beta_mktrf_t9', 'beta_smb_t9', 'beta_hml_t9',
                'beta_rmw_t9', 'beta_cma_t9', 'beta_umd_t9',
                'beta_mktrf_t12', 'beta_smb_t12', 'beta_hml_t12',
                'beta_rmw_t12', 'beta_cma_t12', 'beta_umd_t12'
            ],
            'level': 0.02,
            'reason': '96-99% of values in normal range'
        },
        
        # Standard for other financial variables
        'Standard (1%/99%)': {
            'variables': [
                'excess_return_3mo', 'excess_return_6mo', 'excess_return_9mo', 'excess_return_12mo',
                'calc_log_market_cap', 'calc_roa', 'calc_roe', 'calc_debt_to_assets',
                'calc_debt_to_equity', 'calc_operating_margin', 'calc_profit_margin'
            ],
            'level': 0.01,
            'reason': 'Standard practice for financial variables'
        }
    }
    
    # Create copy for winsorized data
    df_winsorized = df.copy()
    winsorization_report = {}
    
    print("Applying differentiated winsorization:")
    
    # Apply winsorization by strategy
    for strategy_name, strategy_info in winsorization_strategy.items():
        print(f"\n{strategy_name}:")
        print(f"   Level: {strategy_info['level']*100:.0f}%/{100-strategy_info['level']*100:.0f}%")
        print(f"   Reason: {strategy_info['reason']}")
        
        variables_processed = []
        
        for var in strategy_info['variables']:
            if var in df.columns:
                original_data = df[var].dropna()
                
                if len(original_data) > 0:
                    # Apply winsorization
                    winsorized_data = winsorize(original_data, limits=[strategy_info['level'], strategy_info['level']])
                    df_winsorized[var] = df[var].copy()
                    df_winsorized.loc[original_data.index, var] = winsorized_data
                    
                    # Track changes
                    n_changed = (original_data != winsorized_data).sum()
                    pct_changed = (n_changed / len(original_data)) * 100
                    
                    winsorization_report[var] = {
                        'strategy': strategy_name,
                        'level': strategy_info['level'],
                        'n_obs': len(original_data),
                        'n_changed': n_changed,
                        'pct_changed': pct_changed,
                        'original_min': original_data.min(),
                        'original_max': original_data.max(),
                        'winsor_min': winsorized_data.min(),
                        'winsor_max': winsorized_data.max(),
                        'original_std': original_data.std(),
                        'winsor_std': winsorized_data.std()
                    }
                    
                    variables_processed.append(var)
        
        print(f"   Variables processed: {len(variables_processed)}")
        if variables_processed:
            print(f"   Sample: {variables_processed[:3]}{'...' if len(variables_processed) > 3 else ''}")
    
    return df_winsorized, winsorization_report

# apply the enhanced winsorization
df_final, report = enhanced_winsorization_strategy(df)

print("Enhanced winsorization strategy complete")
print("This approach provides:")
print("Variable-specific winsorization levels")
print("5%/95% for problematic variables (p/b)")
print("2%/98% for factor betas")
print("1%/99% for standard financial variables")
print("Quality flags for flexible filtering")
print("Comprehensive before/after analysis")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def create_final_regression_summary_table(df_input, export_to_csv=True, filename="final_regression_summary.csv"):
    """
    Create comprehensive summary statistics table for all regression variables
    after enhanced winsorization - ready for thesis/publication
    """
    
    print("Final regression variables summary statistics")
    print(f"Dataset shape: {df_input.shape}")
    
    # Define all regression variables with proper categorization
    regression_variables = {
        # dependent variables
        'Dependent Variables': {
            'excess_return_3mo': '3-Month Excess Return',
            'excess_return_6mo': '6-Month Excess Return',
            'excess_return_9mo': '9-Month Excess Return', 
            'excess_return_12mo': '12-Month Excess Return'
        },
        
        # control variables  
        'Control Variables': {
            'calc_log_market_cap': 'Log Market Cap (Size)',
            'calc_price_to_book': 'Price-to-Book (Value)',
            'calc_roa': 'ROA (Profitability)'
        },
        
        # factor betas (3-month estimation window)
        'Factor Betas (3-month)': {
            'beta_mktrf_t3': 'Market Beta (MKTRF)',
            'beta_smb_t3': 'Size Beta (SMB)', 
            'beta_hml_t3': 'Value Beta (HML)',
            'beta_rmw_t3': 'Profitability Beta (RMW)',
            'beta_cma_t3': 'Investment Beta (CMA)',
            'beta_umd_t3': 'Momentum Beta (UMD)'
        },
        
        # factor betas (6-month estimation window)
        'Factor Betas (6-month)': {
            'beta_mktrf_t6': 'Market Beta (MKTRF)',
            'beta_smb_t6': 'Size Beta (SMB)',
            'beta_hml_t6': 'Value Beta (HML)', 
            'beta_rmw_t6': 'Profitability Beta (RMW)',
            'beta_cma_t6': 'Investment Beta (CMA)',
            'beta_umd_t6': 'Momentum Beta (UMD)'
        },
        
        # factor betas (9-month estimation window)
        'Factor Betas (9-month)': {
            'beta_mktrf_t9': 'Market Beta (MKTRF)',
            'beta_smb_t9': 'Size Beta (SMB)',
            'beta_hml_t9': 'Value Beta (HML)',
            'beta_rmw_t9': 'Profitability Beta (RMW)',
            'beta_cma_t9': 'Investment Beta (CMA)',
            'beta_umd_t9': 'Momentum Beta (UMD)'
        },
        
        # factor betas (12-month estimation window)  
        'Factor Betas (12-month)': {
            'beta_mktrf_t12': 'Market Beta (MKTRF)',
            'beta_smb_t12': 'Size Beta (SMB)',
            'beta_hml_t12': 'Value Beta (HML)',
            'beta_rmw_t12': 'Profitability Beta (RMW)', 
            'beta_cma_t12': 'Investment Beta (CMA)',
            'beta_umd_t12': 'Momentum Beta (UMD)'
        }
    }
    
    summary_data = []
    for category, variables in regression_variables.items():
        print(f"\n{category.upper()}")
        category_data = []
        for var_code, var_name in variables.items():
            if var_code in df_input.columns:
                data = df_input[var_code].dropna()
                if len(data) > 0:
                    stats = {
                        'Category': category, 'Variable': var_name, 'Code': var_code,
                        'N': len(data), 'Mean': data.mean(), 'Median': data.median(),
                        'Std': data.std(), 'Min': data.min(), 'P25': data.quantile(0.25),
                        'P75': data.quantile(0.75), 'Max': data.max(), 'Skewness': data.skew(),
                        'Kurtosis': data.kurtosis(), 'Missing': len(df_input) - len(data),
                        'Missing_Pct': ((len(df_input) - len(data)) / len(df_input)) * 100
                    }
                    summary_data.append(stats)
                    category_data.append(stats)
        
        if category_data:
            cat_df = pd.DataFrame(category_data)
            print(f"{'Variable':<25} {'N':<6} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8}")
            print("-" * 70)
            for _, row in cat_df.iterrows():
                print(f"{row['Variable'][:24]:<25} {row['N']:<6,.0f} {row['Mean']:<8.3f} {row['Std']:<8.3f} {row['Min']:<8.3f} {row['Max']:<8.3f}")
    
    summary_df = pd.DataFrame(summary_data)
    
    if export_to_csv:
        try:
            summary_df.to_csv(filename, index=False)
            print(f"Exported files:")
            print(f"Detailed summary: {filename}")
        except Exception as e:
            print(f"Export error: {e}")
    
    return summary_df

# run the complete analysis
summary_df = create_final_regression_summary_table(
    df_input=df_final, 
    export_to_csv=True, 
    filename="final_regression_summary.csv"
)

print("All summary tables and exports complete")

In [ ]:
print(df_final.columns.tolist())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def create_averaged_ai_factors(df_final):
    """
    Create averaged AI factors from the three models
    """
    print("Creating averaged AI factors")
    
    ai_factor_mapping = {
        'AI_Washing': 'AI Washing Index', 'Disclosure_Sentiment': 'Disclosure Sentiment',
        'Forward_Looking': 'Forward-Looking', 'Strategic_Depth': 'Strategic Depth',
        'Talent_Investment': 'Talent & Investment', 'Risk_External': 'Risk - External Threats',
        'Risk_Non_Adoption': 'Risk - Non-Adoption', 'Risk_Own_Adoption': 'Risk - Own Adoption'
    }
    models = ['flash1_5', 'flash2_5', 'gpt4o']
    df_with_averages = df_final.copy()
    
    for clean_name, original_name in ai_factor_mapping.items():
        factor_columns = [f"{original_name}_{model}_Numeric" for model in models if f"{original_name}_{model}_Numeric" in df_final.columns]
        if factor_columns:
            df_with_averages[clean_name] = df_final[factor_columns].mean(axis=1)

    available_ai_factors = [name for name in ai_factor_mapping.keys() if name in df_with_averages.columns]
    if available_ai_factors:
        df_with_averages['AI_Cumulative_Score'] = df_with_averages[available_ai_factors].sum(axis=1)
        print("Averaged AI factors and cumulative score created successfully")

    return df_with_averages

def create_final_regression_dataset(df_with_ai):
    """
    Create the final regression dataset, now including t9 betas
    """
    print("Creating final regression dataset")
    
    essential_columns = [
        # identifiers
        'CIK', 'Ticker', 'Year', 'gvkey', 'Company Name', 'Sector',
        # dependent variables
        'excess_return_3mo', 'excess_return_6mo', 'excess_return_9mo', 'excess_return_12mo',
        # control variables
        'calc_log_market_cap', 'calc_price_to_book', 'calc_roa',
        # factor betas t3
        'beta_mktrf_t3', 'beta_smb_t3', 'beta_hml_t3', 'beta_rmw_t3', 'beta_cma_t3', 'beta_umd_t3',
        # factor betas t6
        'beta_mktrf_t6', 'beta_smb_t6', 'beta_hml_t6', 'beta_rmw_t6', 'beta_cma_t6', 'beta_umd_t6',
        # factor betas t9
        'beta_mktrf_t9', 'beta_smb_t9', 'beta_hml_t9', 'beta_rmw_t9', 'beta_cma_t9', 'beta_umd_t9',
        # factor betas t12
        'beta_mktrf_t12', 'beta_smb_t12', 'beta_hml_t12', 'beta_rmw_t12', 'beta_cma_t12', 'beta_umd_t12',
        # averaged ai factors
        'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking', 'Strategic_Depth',
        'Talent_Investment', 'Risk_External', 'Risk_Non_Adoption', 'Risk_Own_Adoption',
        'AI_Cumulative_Score'
    ]
    
    available_columns = [col for col in essential_columns if col in df_with_ai.columns]
    regression_dataset = df_with_ai[available_columns].copy()
    
    print(f"Final regression dataset created with shape: {regression_dataset.shape}")
    return regression_dataset

print("Complete AI factor integration & final data assembly")
print("This script takes df_final (after winsorization), adds the averaged AI scores,")
print("and selects all necessary columns to create the master final_reg_data")

# step 1: create averaged ai factors from the raw scores in df_final
df_with_ai_factors = create_averaged_ai_factors(df_final)

# step 2: assemble the final, complete regression dataset
final_reg_data = create_final_regression_dataset(df_with_ai_factors)

print("Master regression dataframe final_reg_data created successfully")
print("Can now proceed with the regression and diagnostic analysis cells")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def create_portfolio_sorts(df, ai_factors, return_periods=['3mo', '6mo', '9mo', '12mo']):
    """
    create portfolio sorts for ai factors following standard asset pricing methodology
    """
    
    print("Univariate portfolio analysis")
    
    results = {}
    
    for factor in ai_factors:
        if factor not in df.columns:
            print(f"Warning: {factor} not found in dataframe")
            continue
            
        print(f"Analyzing factor: {factor}")
        
        factor_results = {}
        
        # for each return period
        for period in return_periods:
            return_col = f'excess_return_{period}'
            if return_col not in df.columns:
                continue
                
            period_results = {}
            
            # create dataset with complete observations
            analysis_data = df[[factor, return_col, 'Year']].dropna()
            
            if len(analysis_data) < 50:  # minimum observations check
                print(f"   Insufficient data for {period}: {len(analysis_data)} observations")
                continue
            
            # group by year and create quintiles within each year
            yearly_portfolios = []
            
            for year in analysis_data['Year'].unique():
                year_data = analysis_data[analysis_data['Year'] == year].copy()
                
                if len(year_data) < 10:  # need minimum firms per year
                    continue
                
                # create quintiles based on ai factor - handle ties robustly
                try:
                    year_data['quintile'] = pd.qcut(year_data[factor], 
                                                  q=5, 
                                                  labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'],
                                                  duplicates='drop')
                except ValueError:
                    # if qcut fails due to too many duplicates, use rank-based approach
                    year_data['rank'] = year_data[factor].rank(method='first', pct=True)
                    year_data['quintile'] = pd.cut(year_data['rank'], 
                                                 bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                                 labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'],
                                                 include_lowest=True)
                    year_data = year_data.drop('rank', axis=1)
                
                yearly_portfolios.append(year_data)
            
            if not yearly_portfolios:
                continue
                
            # combine all years
            all_portfolios = pd.concat(yearly_portfolios, ignore_index=True)
            
            # calculate portfolio returns
            portfolio_returns = all_portfolios.groupby('quintile')[return_col].agg([
                'mean', 'std', 'count'
            ]).round(4)
            
            # calculate high-minus-low spread (q5 - q1)
            if 'Q5' in portfolio_returns.index and 'Q1' in portfolio_returns.index:
                hml_return = portfolio_returns.loc['Q5', 'mean'] - portfolio_returns.loc['Q1', 'mean']
                
                # get returns for statistical test
                q5_returns = all_portfolios[all_portfolios['quintile'] == 'Q5'][return_col]
                q1_returns = all_portfolios[all_portfolios['quintile'] == 'Q1'][return_col]
                
                # perform t-test for hml spread
                t_stat, p_value = stats.ttest_ind(q5_returns, q1_returns, equal_var=False)
                
                period_results = {
                    'portfolio_returns': portfolio_returns,
                    'hml_spread': hml_return,
                    'hml_tstat': t_stat,
                    'hml_pvalue': p_value,
                    'n_observations': len(all_portfolios)
                }
                
                print(f"   {period}: hml = {hml_return:.4f} (t={t_stat:.2f}, p={p_value:.3f}), n={len(all_portfolios)}")
            
            factor_results[period] = period_results
        
        results[factor] = factor_results
    
    return results

def run_complete_univariate_analysis(final_reg_data):
    """
    run the complete univariate analysis as specified in the thesis methodology
    """
    
    print("Running complete univariate analysis")
    print("Following the methodology outlined in section 3.5 of the thesis...")
    
    # define ai factors to analyze
    ai_factors = [
        'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking', 'Strategic_Depth',
        'Talent_Investment', 'Risk_External', 'Risk_Non_Adoption', 'Risk_Own_Adoption',
        'AI_Cumulative_Score'
    ]
    
    # check data availability
    print(f"Dataset overview:")
    print(f"   Total observations: {len(final_reg_data)}")
    print(f"   Years covered: {final_reg_data['Year'].min()} - {final_reg_data['Year'].max()}")
    print(f"   Unique firms: {final_reg_data['CIK'].nunique()}")
    
    # data quality check for ai factors
    print("AI factors data quality check:")
    for factor in ai_factors:
        if factor in final_reg_data.columns:
            factor_data = final_reg_data[factor].dropna()
            n_unique = factor_data.nunique()
            n_total = len(factor_data)
            print(f"   {factor}: {n_total} obs, {n_unique} unique values ({n_unique/n_total:.3f} ratio)")
        else:
            print(f"   {factor}: not found in dataset")
    
    # 1. portfolio sorts analysis
    portfolio_results = create_portfolio_sorts(final_reg_data, ai_factors)
    
    print("Univariate analysis complete")
    print("Next step: proceed with multivariate regression analysis")
    
    return portfolio_results

# run the analysis
portfolio_results = run_complete_univariate_analysis(final_reg_data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def create_sector_neutral_sorts(df, ai_factors, return_periods=['12mo'], min_firms_per_tercile=5):
    """
    create sector-neutral portfolio sorts using terciles within each sector
    """
    
    print("Sector-neutral portfolio analysis")
    print("Using terciles within sectors to control for industry effects")
    
    sector_results = {}
    
    for factor in ai_factors:
        if factor not in df.columns:
            continue
            
        print(f"Analyzing factor: {factor}")
        
        factor_results = {}
        
        for period in return_periods:
            return_col = f'excess_return_{period}'
            if return_col not in df.columns:
                continue
                
            # get complete data
            analysis_data = df[[factor, return_col, 'Sector', 'Year']].dropna()
            
            if len(analysis_data) < 50:
                continue
            
            sector_spreads = []
            sector_summary = []
            
            # analyze each sector separately
            for sector in analysis_data['Sector'].unique():
                sector_data = analysis_data[analysis_data['Sector'] == sector].copy()
                
                if len(sector_data) < min_firms_per_tercile * 3:  # need minimum firms for terciles
                    continue
                
                # create terciles within sector - handle ties robustly
                try:
                    sector_data['tercile'] = pd.qcut(sector_data[factor], 
                                                   q=3, 
                                                   labels=['Low', 'Mid', 'High'],
                                                   duplicates='drop')
                except ValueError:
                    # handle ties with rank-based approach
                    sector_data['rank'] = sector_data[factor].rank(method='first', pct=True)
                    sector_data['tercile'] = pd.cut(sector_data['rank'], 
                                                  bins=[0, 0.33, 0.67, 1.0],
                                                  labels=['Low', 'Mid', 'High'],
                                                  include_lowest=True)
                    sector_data = sector_data.drop('rank', axis=1)
                
                # calculate tercile returns
                tercile_returns = sector_data.groupby('tercile')[return_col].agg([
                    'mean', 'std', 'count'
                ])
                
                if 'High' in tercile_returns.index and 'Low' in tercile_returns.index:
                    # high-minus-low spread
                    hml_spread = tercile_returns.loc['High', 'mean'] - tercile_returns.loc['Low', 'mean']
                    
                    # get individual returns for t-test
                    high_returns = sector_data[sector_data['tercile'] == 'High'][return_col]
                    low_returns = sector_data[sector_data['tercile'] == 'Low'][return_col]
                    
                    # perform t-test
                    if len(high_returns) > 1 and len(low_returns) > 1:
                        t_stat, p_value = stats.ttest_ind(high_returns, low_returns, equal_var=False)
                        
                        sector_spreads.append(hml_spread)
                        sector_summary.append({
                            'Sector': sector,
                            'HML_Spread': hml_spread,
                            'T_Stat': t_stat,
                            'P_Value': p_value,
                            'N_Firms': len(sector_data),
                            'N_High': len(high_returns),
                            'N_Low': len(low_returns)
                        })
                        
                        # add significance stars
                        if p_value < 0.01:
                            sig_stars = '***'
                        elif p_value < 0.05:
                            sig_stars = '**'
                        elif p_value < 0.10:
                            sig_stars = '*'
                        else:
                            sig_stars = ''
                        
                        print(f"   {sector}: {hml_spread:.4f}{sig_stars} (t={t_stat:.2f}, n={len(sector_data)})")
            
            # overall sector-neutral analysis
            if sector_spreads:
                overall_spread = np.mean(sector_spreads)
                spread_std = np.std(sector_spreads, ddof=1)
                n_sectors = len(sector_spreads)
                
                # t-test for overall sector-neutral spread
                if n_sectors > 1:
                    overall_t_stat = overall_spread / (spread_std / np.sqrt(n_sectors))
                    overall_p_value = 2 * (1 - stats.t.cdf(abs(overall_t_stat), n_sectors - 1))
                    
                    print(f"Sector-neutral overall: {overall_spread:.4f} (t={overall_t_stat:.2f}, p={overall_p_value:.3f})")
                    print(f"Based on {n_sectors} sectors")
                
                factor_results[period] = {
                    'sector_summary': pd.DataFrame(sector_summary),
                    'overall_spread': overall_spread,
                    'overall_t_stat': overall_t_stat,
                    'overall_p_value': overall_p_value,
                    'n_sectors': n_sectors
                }
        
        sector_results[factor] = factor_results
    
    return sector_results

def create_pooled_sector_neutral_portfolio(df, factor='AI_Cumulative_Score', return_period='12mo'):
    """
    create pooled high/low ai portfolios balanced across sectors
    """
    
    print(f"Pooled sector-neutral analysis: {factor}")
    
    return_col = f'excess_return_{return_period}'
    analysis_data = df[[factor, return_col, 'Sector', 'Year', 'CIK']].dropna()
    
    pooled_high = []
    pooled_low = []
    sector_contributions = []
    
    for sector in analysis_data['Sector'].unique():
        sector_data = analysis_data[analysis_data['Sector'] == sector].copy()
        
        if len(sector_data) < 9:  # need minimum for terciles
            continue
        
        # create terciles within sector
        try:
            sector_data['tercile'] = pd.qcut(sector_data[factor], 
                                           q=3, 
                                           labels=['Low', 'Mid', 'High'],
                                           duplicates='drop')
        except ValueError:
            # handle ties with rank-based approach
            sector_data['rank'] = sector_data[factor].rank(method='first', pct=True)
            sector_data['tercile'] = pd.cut(sector_data['rank'], 
                                          bins=[0, 0.33, 0.67, 1.0],
                                          labels=['Low', 'Mid', 'High'],
                                          include_lowest=True)
            sector_data = sector_data.drop('rank', axis=1)
        
        # get high and low tercile firms
        high_firms = sector_data[sector_data['tercile'] == 'High']
        low_firms = sector_data[sector_data['tercile'] == 'Low']
        
        if len(high_firms) > 0 and len(low_firms) > 0:
            pooled_high.extend(high_firms[return_col].tolist())
            pooled_low.extend(low_firms[return_col].tolist())
            
            sector_contributions.append({
                'Sector': sector,
                'N_High': len(high_firms),
                'N_Low': len(low_firms),
                'High_Mean': high_firms[return_col].mean(),
                'Low_Mean': low_firms[return_col].mean(),
                'Sector_Spread': high_firms[return_col].mean() - low_firms[return_col].mean()
            })
    
    # perform pooled t-test
    if len(pooled_high) > 0 and len(pooled_low) > 0:
        pooled_high = np.array(pooled_high)
        pooled_low = np.array(pooled_low)
        
        hml_spread = pooled_high.mean() - pooled_low.mean()
        t_stat, p_value = stats.ttest_ind(pooled_high, pooled_low, equal_var=False)
        
        print(f"Pooled portfolio results:")
        print(f"   High AI portfolio: {pooled_high.mean():.4f} ({len(pooled_high)} firms)")
        print(f"   Low AI portfolio:  {pooled_low.mean():.4f} ({len(pooled_low)} firms)")
        print(f"   High-minus-low:    {hml_spread:.4f} (t={t_stat:.2f}, p={p_value:.3f})")
        
        # significance stars
        if p_value < 0.01:
            sig_stars = '***'
        elif p_value < 0.05:
            sig_stars = '**'
        elif p_value < 0.10:
            sig_stars = '*'
        else:
            sig_stars = ''
        
        print(f"   Final result: {hml_spread:.4f}{sig_stars}")
        
        # show sector contributions
        contrib_df = pd.DataFrame(sector_contributions)
        print(f"Sector contributions:")
        print(contrib_df.round(4))
        
        return {
            'hml_spread': hml_spread,
            't_stat': t_stat,
            'p_value': p_value,
            'n_high': len(pooled_high),
            'n_low': len(pooled_low),
            'high_mean': pooled_high.mean(),
            'low_mean': pooled_low.mean(),
            'sector_contributions': contrib_df
        }
    
    return None

def create_sector_summary_table(sector_results, focus_factor='AI_Cumulative_Score', period='12mo'):
    """
    create summary table matching table 12 in the thesis
    """
    
    print(f"Sector summary table - {focus_factor}")
    
    if focus_factor not in sector_results or period not in sector_results[focus_factor]:
        print("Data not available for summary table")
        return None
    
    sector_data = sector_results[focus_factor][period]['sector_summary']
    
    # format the summary table
    summary_table = sector_data.copy()
    summary_table['HML_Spread_Pct'] = summary_table['HML_Spread'] * 100  # convert to percentage
    
    # add significance stars
    def add_stars(row):
        if row['P_Value'] < 0.01:
            return f"{row['HML_Spread_Pct']:.2f}***"
        elif row['P_Value'] < 0.05:
            return f"{row['HML_Spread_Pct']:.2f}**"
        elif row['P_Value'] < 0.10:
            return f"{row['HML_Spread_Pct']:.2f}*"
        else:
            return f"{row['HML_Spread_Pct']:.2f}"
    
    summary_table['Spread_Display'] = summary_table.apply(add_stars, axis=1)
    
    # create final display table
    final_table = summary_table[['Sector', 'Spread_Display', 'T_Stat', 'P_Value', 'N_Firms']].copy()
    final_table.columns = ['Sector', 'High-Low Spread (%)', 'T-Stat', 'P-Value', 'N Firms']
    final_table = final_table.round({'T-Stat': 2, 'P-Value': 3})
    
    print(final_table.to_string(index=False))
    
    return final_table

def plot_sector_results(sector_results, focus_factor='AI_Cumulative_Score', period='12mo'):
    """
    visualize sector-specific results
    """
    
    if focus_factor not in sector_results or period not in sector_results[focus_factor]:
        return
    
    sector_data = sector_results[focus_factor][period]['sector_summary']
    
    # create bar plot of sector spreads
    plt.figure(figsize=(12, 8))
    
    colors = ['red' if x < 0 else 'blue' for x in sector_data['HML_Spread']]
    bars = plt.bar(range(len(sector_data)), sector_data['HML_Spread'] * 100, color=colors, alpha=0.7)
    
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    plt.xlabel('Sector')
    plt.ylabel('High-minus-low spread (%)')
    plt.title(f'Sector-specific AI factor effects ({focus_factor})')
    plt.xticks(range(len(sector_data)), sector_data['Sector'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # add significance indicators
    for i, (_, row) in enumerate(sector_data.iterrows()):
        if row['P_Value'] < 0.01:
            plt.text(i, row['HML_Spread'] * 100 + 0.5, '***', ha='center', fontsize=12)
        elif row['P_Value'] < 0.05:
            plt.text(i, row['HML_Spread'] * 100 + 0.5, '**', ha='center', fontsize=12)
        elif row['P_Value'] < 0.10:
            plt.text(i, row['HML_Spread'] * 100 + 0.5, '*', ha='center', fontsize=12)
    
    plt.tight_layout()
    plt.show()

def run_complete_sector_analysis(final_reg_data):
    """
    run the complete sector-specific analysis as outlined in section 4.1.2
    """
    
    print("Running complete sector-specific analysis")
    print("Following methodology from section 4.1.2: controlling for sectors")
    
    # check sector distribution
    print(f"Sector distribution:")
    sector_counts = final_reg_data['Sector'].value_counts()
    print(sector_counts)
    
    # define key ai factors for sector analysis
    ai_factors = ['AI_Cumulative_Score']  # start with main factor
    
    # 1. sector-neutral sorts for all factors
    sector_results = create_sector_neutral_sorts(final_reg_data, ai_factors)
    
    # 2. pooled sector-neutral portfolio analysis (main test)
    pooled_results = create_pooled_sector_neutral_portfolio(final_reg_data, 
                                                           factor='AI_Cumulative_Score',
                                                           return_period='12mo')
    
    # 3. create summary table (table 12 equivalent)
    summary_table = create_sector_summary_table(sector_results, 
                                               focus_factor='AI_Cumulative_Score', 
                                               period='12mo')
    
    # 4. visualization
    plot_sector_results(sector_results, focus_factor='AI_Cumulative_Score', period='12mo')
    
    print("Sector-specific analysis complete!")
    print("   Key finding: effects persist even within sectors (sector-neutral)")
    
    return sector_results, pooled_results, summary_table

# run the analysis
sector_results, pooled_results, summary_table = run_complete_sector_analysis(final_reg_data)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def run_academic_regression_table(reg_data, ai_factor, dependent_var, sector_col, beta_horizon='t12'):
    """
    run regression analysis and return results in academic table format
    """
    
    factor_models = {
        'Base': [],
        'FF3': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}'],
        'FF5': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}', 
                f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}'],
        'FF6': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}', 
                f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}', f'beta_umd_{beta_horizon}']
    }
    
    base_formula = f"{dependent_var} ~ {ai_factor}"
    if sector_col and sector_col in reg_data.columns:
        base_formula += f" + C({sector_col})"
    if 'Year' in reg_data.columns:
        base_formula += f" + C(Year)"
    
    results = {}
    
    for model_name, factors in factor_models.items():
        available_factors = [f for f in factors if f in reg_data.columns]
        
        formula = base_formula
        if available_factors:
            formula += " + " + " + ".join(available_factors)
        
        try:
            model = smf.ols(formula, data=reg_data).fit()
            
            ai_coef = model.params.get(ai_factor, np.nan)
            ai_stderr = model.bse.get(ai_factor, np.nan)
            ai_tstat = model.tvalues.get(ai_factor, np.nan)
            ai_pvalue = model.pvalues.get(ai_factor, np.nan)
            
            results[model_name] = {
                'model': model, 'ai_coefficient': ai_coef, 'ai_stderr': ai_stderr,
                'ai_tstat': ai_tstat, 'ai_pvalue': ai_pvalue,
                'r_squared': model.rsquared, 'adj_r_squared': model.rsquared_adj,
                'n_obs': model.nobs, 'available_factors': available_factors
            }
            
        except Exception as e:
            print(f"Error running regression for model '{model_name}'. Formula: '{formula}'. Error: {e}")
            results[model_name] = None
    
    return results

def run_all_academic_tables(reg_data, ai_factors, export_tables=True):
    """
    main orchestrator function. runs regression analysis for all ai factors
    and exports the results
    """
    
    print("Creating academic regression tables")
    print("Using pre-computed AI factors and cumulative score from the master dataset")
    
    all_ai_factors_to_test = ai_factors + ['AI_Cumulative_Score']
    
    horizons = {
        't3': 'excess_return_3mo', 't6': 'excess_return_6mo',
        't9': 'excess_return_9mo', 't12': 'excess_return_12mo'
    }
    
    all_tables = {}
    
    for ai_factor in all_ai_factors_to_test:
        if ai_factor not in reg_data.columns:
            print(f"'{ai_factor}' not found, skipping...")
            continue
            
        print(f"Creating tables for: {ai_factor.replace('_', ' ').title()}")
        factor_tables = {}
        
        for horizon, dependent_var in horizons.items():
            if dependent_var not in reg_data.columns:
                print(f"  Dependent var '{dependent_var}' not found, skipping {horizon}")
                continue
                
            print(f"  Running for {horizon} horizon ({dependent_var})")
            
            results = run_academic_regression_table(
                reg_data, ai_factor, dependent_var, 'Sector', horizon
            )
            
            factor_tables[horizon] = {
                'results': results,
                'dependent_var': dependent_var
            }
            
        all_tables[ai_factor] = factor_tables
    
    return all_tables, reg_data

# 1. define the list of individual, averaged ai factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# 2. run the complete analysis using the master dataframe
all_tables, final_data_for_reg = run_all_academic_tables(final_reg_data, ai_factors)

print("Academic regression analysis complete")

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# diagnostic functions
# This section defines all the necessary tools for the analysis.

def calculate_vif_individual_regressions(reg_data, ai_factors, ff_factors, horizon='t12'):
    """
    Calculate VIF for individual AI factor regressions.
    """
    print(f"VIF analysis for individual AI factor regressions (horizon: {horizon.upper()})")
    print("=" * 60)
    
    available_ff_factors = [f for f in ff_factors if f in reg_data.columns]
    available_ai_factors = [f for f in ai_factors if f in reg_data.columns]
    
    all_vif_results = []
    
    for ai_factor in available_ai_factors:
        factors_for_this_regression = [ai_factor] + available_ff_factors
        vif_data = reg_data[factors_for_this_regression].dropna()
        
        if len(vif_data) < 50:
            print(f"  Skipping VIF for {ai_factor} due to insufficient data ({len(vif_data)} obs).")
            continue
        
        for i, factor in enumerate(factors_for_this_regression):
            try:
                vif_value = variance_inflation_factor(vif_data.values, i)
                all_vif_results.append({
                    'AI_Factor_In_Model': ai_factor, 'Variable': factor,
                    'VIF': vif_value, 'Concern': 'High' if vif_value > 10 else 'Moderate' if vif_value > 5 else 'Low'
                })
            except Exception as e:
                print(f"  Error calculating VIF for {factor}: {e}")
        
    vif_df = pd.DataFrame(all_vif_results)
    
    if not vif_df.empty:
        max_vif = vif_df['VIF'].max()
        print(f"VIF analysis complete for {horizon.upper()}. Max VIF observed: {max_vif:.2f}")
        if max_vif > 10:
            print("  High multicollinearity detected in at least one specification.")
        else:
            print("  No concerning multicollinearity in any individual regression specification.")
            
    return vif_df

def run_diagnostic_tests(model_results, test_name=""):
    """
    Run a suite of diagnostic tests on a fitted regression model.
    """
    try:
        model = model_results['model']
        residuals = model.resid
        
        bp_stat, bp_pvalue, _, _ = het_breuschpagan(residuals, model.model.exog)
        white_stat, white_pvalue, _, _ = het_white(residuals, model.model.exog)
        dw_stat = durbin_watson(residuals)
        jb_stat, jb_pvalue = stats.jarque_bera(residuals)
        
        return {
            'Test_Name': test_name, 'BP_pvalue': bp_pvalue, 'White_pvalue': white_pvalue,
            'Durbin_Watson': dw_stat, 'JB_pvalue': jb_pvalue
        }
    except Exception as e:
        print(f"Error in diagnostic tests for '{test_name}': {e}")
        return None

def run_comprehensive_diagnostics(reg_data, ai_factors, horizon, dependent_var):
    """
    Run a full suite of diagnostics for a specific regression setup.
    """
    print(f"\nRunning comprehensive diagnostics (horizon: {horizon.upper()})")
    print("=" * 60)
    
    ff_factors = [f'beta_mktrf_{horizon}', f'beta_smb_{horizon}', f'beta_hml_{horizon}',
                  f'beta_rmw_{horizon}', f'beta_cma_{horizon}', f'beta_umd_{horizon}']
    
    vif_results = calculate_vif_individual_regressions(reg_data, ai_factors, ff_factors, horizon)
    
    print("\nRunning diagnostics on AI cumulative score model...")
    all_diagnostics = []
    if 'AI_Cumulative_Score' in reg_data.columns:
        base_formula = f"{dependent_var} ~ AI_Cumulative_Score + C(Sector) + C(Year)"
        available_ff = [f for f in ff_factors if f in reg_data.columns]
        
        ff6_formula = base_formula
        if available_ff:
            ff6_formula += " + " + " + ".join(available_ff)
        
        try:
            base_model = smf.ols(base_formula, data=reg_data).fit()
            all_diagnostics.append(run_diagnostic_tests({'model': base_model}, "Cumulative Score - Base Model"))
            
            if available_ff:
                ff6_model = smf.ols(ff6_formula, data=reg_data).fit()
                all_diagnostics.append(run_diagnostic_tests({'model': ff6_model}, "Cumulative Score - FF6 Model"))

        except Exception as e:
            print(f"Error running diagnostic models for {horizon.upper()}: {e}")
            
    recommendations = []
    if not vif_results.empty and vif_results['VIF'].max() > 10:
        recommendations.append("VIF > 10 detected. Check VIF report.")
    else:
        recommendations.append("Multicollinearity is not a concern in individual regressions.")

    # Check for heteroskedasticity in the first valid diagnostic result
    het_detected = False
    for diag_result in all_diagnostics:
        if diag_result and diag_result.get('BP_pvalue', 1) < 0.05:
            het_detected = True
            break
            
    if het_detected:
        recommendations.append("Heteroskedasticity detected (p < 0.05). Recommendation: use robust standard errors, e.g., .fit(cov_type='HC3').")
    else:
        recommendations.append("No significant heteroskedasticity detected.")
        
    print("\nDiagnostic summary & recommendations")
    print("-" * 50)
    for rec in recommendations:
        print(rec)
        
    return {'vif_results': vif_results, 'model_diagnostics': [d for d in all_diagnostics if d is not None]}

def export_diagnostic_results(diagnostic_results, horizon):
    """
    Exports comprehensive results for all tests.
    """
    print(f"\nExporting diagnostic results for {horizon.upper()}")
    print("=" * 45)
    
    try:
        if diagnostic_results.get('vif_results') is not None and not diagnostic_results['vif_results'].empty:
            vif_file = f"diagnostics_vif_analysis_{horizon}.csv"
            diagnostic_results['vif_results'].to_csv(vif_file, index=False)
            print(f"VIF results exported to: {vif_file}")
            
        if diagnostic_results.get('model_diagnostics'):
            diag_df = pd.DataFrame(diagnostic_results['model_diagnostics'])
            diag_file = f"diagnostics_model_summary_{horizon}.csv"
            diag_df.to_csv(diag_file, index=False)
            print(f"Model diagnostics (BP, White, etc.) exported to: {diag_file}")

    except Exception as e:
        print(f"An error occurred during export for {horizon.upper()}: {e}")

# 1. Define AI factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# 2. Define the horizons to test
horizons_to_test = {
    't3': 'excess_return_3mo', 't6': 'excess_return_6mo',
    't9': 'excess_return_9mo', 't12': 'excess_return_12mo'
}

# 3. Loop through, run diagnostics, and export for each horizon
all_diagnostics_results = {}
for horizon, dependent_var in horizons_to_test.items():
    if dependent_var in final_reg_data.columns:
        diagnostics = run_comprehensive_diagnostics(
            final_reg_data, ai_factors, horizon, dependent_var
        )
        export_diagnostic_results(diagnostics, horizon)
        all_diagnostics_results[horizon] = diagnostics
    else:
        print(f"\nSkipping horizon {horizon.upper()} because dependent variable '{dependent_var}' is not in the data.")

print("\n\nAll diagnostic tests and exports complete for all horizons!")

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def run_academic_regression_table(reg_data, ai_factor, dependent_var, sector_col, beta_horizon='t12', cluster_col='Ticker'):
    """
    run regression using firm-clustered standard errors
    manually handles missing data before fitting to prevent errors
    """
    
    factor_models = {
        'Base': [],
        'FF3': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}'],
        'FF5': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}', 
                f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}'],
        'FF6': [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}', 
                f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}', f'beta_umd_{beta_horizon}']
    }
    
    base_formula = f"{dependent_var} ~ {ai_factor}"
    if sector_col and sector_col in reg_data.columns:
        base_formula += f" + C({sector_col})"
    if 'Year' in reg_data.columns:
        base_formula += f" + C(Year)"
    
    results = {}
    
    if not cluster_col or cluster_col not in reg_data.columns:
        print(f"Error: cluster column '{cluster_col}' not found. Aborting")
        return None

    for model_name, factors in factor_models.items():
        formula = base_formula
        available_factors = [f for f in factors if f in reg_data.columns]
        if available_factors:
            formula += " + " + " + ".join(available_factors)

        # manually prepare the data to handle missing values before fitting
        # 1. identify all variables needed for this specific regression
        all_vars_in_formula = [dependent_var, ai_factor, sector_col, 'Year', cluster_col] + available_factors
        
        # 2. create a clean subset of the data with no missing values for these variables
        clean_data = reg_data[all_vars_in_formula].dropna()

        if len(clean_data) < 20: # check if there's enough data to run a regression
            print(f"Skipping model '{model_name}' due to insufficient data after dropping NaNs ({len(clean_data)} obs)")
            results[model_name] = None
            continue

        try:
            # 3. run the model on the clean data and provide the perfectly aligned cluster groups
            model = smf.ols(formula, data=clean_data).fit(
                cov_type='cluster', 
                cov_kwds={'groups': clean_data[cluster_col]}
            )
            
            ai_coef = model.params.get(ai_factor, np.nan)
            ai_stderr = model.bse.get(ai_factor, np.nan)
            ai_tstat = model.tvalues.get(ai_factor, np.nan)
            ai_pvalue = model.pvalues.get(ai_factor, np.nan)
            
            factor_stats = {}
            for factor in available_factors:
                factor_stats[factor] = {
                    'coef': model.params.get(factor, np.nan),
                    'stderr': model.bse.get(factor, np.nan),
                    'tstat': model.tvalues.get(factor, np.nan),
                    'pvalue': model.pvalues.get(factor, np.nan)
                }
            
            results[model_name] = {
                'model': model, 'ai_coefficient': ai_coef, 'ai_stderr': ai_stderr,
                'ai_tstat': ai_tstat, 'ai_pvalue': ai_pvalue, 'factor_stats': factor_stats,
                'r_squared': model.rsquared, 'adj_r_squared': model.rsquared_adj,
                'n_obs': model.nobs, 'available_factors': available_factors
            }
        except Exception as e:
            print(f"Error running regression for model '{model_name}'. Formula: '{formula}'. Error: {e}")
            results[model_name] = None
    
    return results

def format_coefficient_academic(coef, stderr, pvalue):
    """
    format a coefficient and its standard error, adding significance stars
    """
    if pd.isna(coef) or pd.isna(stderr):
        return "", ""
    
    if pvalue < 0.01: stars = "***"
    elif pvalue < 0.05: stars = "**"
    elif pvalue < 0.1: stars = "*"
    else: stars = ""
    
    coef_str = f"{coef:.4f}{stars}"
    stderr_str = f"({stderr:.4f})"
    
    return coef_str, stderr_str

def create_academic_table_single_factor(reg_data, ai_factor, beta_horizon, dependent_var, sector_col='Sector', cluster_col='Ticker'):
    """
    assemble the full academic-style regression table for one ai factor
    """
    results = run_academic_regression_table(reg_data, ai_factor, dependent_var, sector_col, beta_horizon, cluster_col)
    
    if results is None:
        return pd.DataFrame(), None

    variable_order = [
        ai_factor, f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}',
        f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}', f'beta_umd_{beta_horizon}'
    ]
    
    variable_names = {
        ai_factor: ai_factor.replace('_', ' ').title(),
        f'beta_mktrf_{beta_horizon}': 'Market Beta', f'beta_smb_{beta_horizon}': 'Size Beta',
        f'beta_hml_{beta_horizon}': 'Value Beta', f'beta_rmw_{beta_horizon}': 'Profitability Beta',
        f'beta_cma_{beta_horizon}': 'Investment Beta', f'beta_umd_{beta_horizon}': 'Momentum Beta'
    }
    
    table_data = []
    for var in variable_order:
        row_data = {'Variable': variable_names.get(var, var)}
        for model_name in ['Base', 'FF3', 'FF5', 'FF6']:
            if results.get(model_name) is None:
                row_data[f'{model_name}_coef'], row_data[f'{model_name}_stderr'] = "", ""
                continue
                
            if var == ai_factor:
                coef, stderr, pvalue = results[model_name]['ai_coefficient'], results[model_name]['ai_stderr'], results[model_name]['ai_pvalue']
            else:
                factor_stats = results[model_name]['factor_stats']
                if var in factor_stats:
                    coef, stderr, pvalue = factor_stats[var]['coef'], factor_stats[var]['stderr'], factor_stats[var]['pvalue']
                else:
                    coef = stderr = pvalue = np.nan
            
            coef_str, stderr_str = format_coefficient_academic(coef, stderr, pvalue)
            row_data[f'{model_name}_coef'] = coef_str
            row_data[f'{model_name}_stderr'] = stderr_str
            
        table_data.append(row_data)
    
    summary_rows = []
    for stat_name in ['R²', 'Adj. R²', 'Observations']:
        row = {'Variable': stat_name}
        for model_name in ['Base', 'FF3', 'FF5', 'FF6']:
            if results.get(model_name) is not None:
                if stat_name == 'R²': value = f"{results[model_name]['r_squared']:.4f}"
                elif stat_name == 'Adj. R²': value = f"{results[model_name]['adj_r_squared']:.4f}"
                else: value = f"{int(results[model_name]['n_obs']):,}"
                row[f'{model_name}_coef'] = value
                row[f'{model_name}_stderr'] = ""
            else:
                row[f'{model_name}_coef'], row[f'{model_name}_stderr'] = "", ""
        summary_rows.append(row)
        
    return pd.DataFrame(table_data + summary_rows), results

def run_all_academic_tables(reg_data, ai_factors, export_tables=True, cluster_col='Ticker'):
    """
    main orchestrator function. runs regression analysis and exports the results
    using firm-clustered standard errors
    """
    
    print("Creating academic regression tables with firm-clustered standard errors")
    print("=" * 75)
    print(f"Clustering standard errors by firm identifier: '{cluster_col}'")
    
    all_ai_factors_to_test = ai_factors + ['AI_Cumulative_Score']
    
    horizons = {
        't3': 'excess_return_3mo', 't6': 'excess_return_6mo',
        't9': 'excess_return_9mo', 't12': 'excess_return_12mo'
    }
    
    all_tables = {}
    
    for ai_factor in all_ai_factors_to_test:
        if ai_factor not in reg_data.columns:
            print(f"'{ai_factor}' not found, skipping...")
            continue
            
        print(f"\nCreating tables for: {ai_factor.replace('_', ' ').title()}")
        factor_tables = {}
        
        for horizon, dependent_var in horizons.items():
            if dependent_var not in reg_data.columns:
                print(f"  Dependent var '{dependent_var}' not found, skipping {horizon}")
                continue
                
            print(f"  Running for {horizon} horizon ({dependent_var})")
            
            table_df, results = create_academic_table_single_factor(
                reg_data, ai_factor, horizon, dependent_var, cluster_col=cluster_col
            )
            
            if results: 
                factor_tables[horizon] = {
                    'table_df': table_df, 'results': results, 'dependent_var': dependent_var
                }
            
        all_tables[ai_factor] = factor_tables
    
    if export_tables:
        export_academic_tables(all_tables)
    
    return all_tables, reg_data

def export_academic_tables(all_tables):
    """
    exports regression results into two formats: a detailed summary and individual tables
    """
    print(f"\nExporting regression results")
    print("=" * 35)
    
    print("1. Exporting detailed summary of all models...")
    summary_data = []
    for ai_factor, factor_tables in all_tables.items():
        for horizon, table_info in factor_tables.items():
            for model_name, result in table_info['results'].items():
                if result:
                    summary_data.append({
                        'AI_Factor': ai_factor, 'Horizon': horizon,
                        'Dependent_Variable': table_info['dependent_var'], 'Model': model_name,
                        'AI_Coefficient': result['ai_coefficient'], 'AI_Std_Error': result['ai_stderr'],
                        'AI_t_stat': result['ai_tstat'], 'AI_p_value': result['ai_pvalue'],
                        'R_Squared': result['r_squared'], 'Adj_R_Squared': result['adj_r_squared'],
                        'N_Obs': result['n_obs']
                    })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('academic_regression_summary_all_results_CLUSTERED.csv', index=False)
    print(f"Detailed summary exported to: academic_regression_summary_all_results_CLUSTERED.csv")

    print("\n2. Exporting individual, publication-ready academic tables...")
    exported_count = 0
    for ai_factor, factor_tables in all_tables.items():
        for horizon, table_info in factor_tables.items():
            table_df = table_info['table_df']
            
            output_rows = [['Variable', 'Base Model', 'FF3 Model', 'FF5 Model', 'FF6 Model']]
            for _, row in table_df.iterrows():
                output_rows.append([
                    row['Variable'], row['Base_coef'], row['FF3_coef'],
                    row['FF5_coef'], row['FF6_coef']
                ])
                if any(row[f'{m}_stderr'] for m in ['Base', 'FF3', 'FF5', 'FF6']):
                    output_rows.append([
                        '', row['Base_stderr'], row['FF3_stderr'],
                        row['FF5_stderr'], row['FF6_stderr']
                    ])

            export_df = pd.DataFrame(output_rows[1:], columns=output_rows[0])
            export_df.to_csv(f"table_{ai_factor}_{horizon}_CLUSTERED.csv", index=False)
            exported_count += 1
            
    print(f"All {exported_count} individual academic tables exported successfully")


def display_sample_table(all_tables, ai_factor='AI_Cumulative_Score', horizon='t12'):
    """
    display a single, formatted sample table in the console for a quick check
    """
    if ai_factor in all_tables and horizon in all_tables[ai_factor]:
        table_info = all_tables[ai_factor][horizon]
        table_df = table_info['table_df']
        
        print(f"\n\nSample table (with firm-clustered SE): {ai_factor.replace('_', ' ').title()} ({horizon})")
        print(f"Dependent variable: {table_info['dependent_var']}")
        print("=" * 80)
        print(f"{'Variable':<20} {'Base':<15} {'FF3':<15} {'FF5':<15} {'FF6':<15}")
        print("-" * 80)
        
        for _, row in table_df.iterrows():
            var_name = row['Variable'][:19]
            print(f"{var_name:<20} {row['Base_coef']:<15} {row['FF3_coef']:<15} {row['FF5_coef']:<15} {row['FF6_coef']:<15}")
            if any(row[f'{m}_stderr'] for m in ['Base', 'FF3', 'FF5', 'FF6']):
                print(f"{'':<20} {row['Base_stderr']:<15} {row['FF3_stderr']:<15} {row['FF5_stderr']:<15} {row['FF6_stderr']:<15}")
        
        print("\nNotes: firm-clustered standard errors (by ticker) in parentheses. *p<0.1, **p<0.05, ***p<0.01")

print("Academic regression table generator (firm-clustered SE version)")
print("=" * 65)
print("This is the final step. It takes the master 'final_reg_data' and runs all regressions")
print("It now uses firm-clustered standard errors for valid inference in panel data")

# 1. define the list of individual, averaged ai factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# 2. run the complete analysis using the master dataframe ('final_reg_data')
all_tables, final_data_for_reg = run_all_academic_tables(final_reg_data, ai_factors, cluster_col='Ticker')

# 3. display a sample table to check the results for the main hypothesis
display_sample_table(all_tables, ai_factor='AI_Cumulative_Score', horizon='t12')

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# function definitions
# Functions are adapted from the Fama-French script to handle book controls.

def run_book_controls_regression_table(reg_data, ai_factor, dependent_var, sector_col, cluster_col='Ticker'):
    """
    Runs regressions for a progressive buildup of book control variables.
    This function is analogous to 'run_academic_regression_table' but for book controls.
    It uses firm-clustered standard errors and handles missing data.
    """
    
    # Define the progressive models, adding one control at a time
    control_models = {
        'Base': [],
        'Base + Size': ['calc_log_market_cap'],
        'Base + Size + Value': ['calc_log_market_cap', 'calc_price_to_book'],
        'Full Controls': ['calc_log_market_cap', 'calc_price_to_book', 'calc_roa']
    }
    
    # Base formula includes the AI factor and fixed effects (Sector, Year)
    base_formula = f"{dependent_var} ~ {ai_factor}"
    if sector_col and sector_col in reg_data.columns:
        base_formula += f" + C({sector_col})"
    if 'Year' in reg_data.columns:
        base_formula += f" + C(Year)"
    
    results = {}
    
    if not cluster_col or cluster_col not in reg_data.columns:
        print(f"Error: Cluster column '{cluster_col}' not found. Aborting.")
        return None

    for model_name, controls in control_models.items():
        formula = base_formula
        available_controls = [c for c in controls if c in reg_data.columns]
        if available_controls:
            formula += " + " + " + ".join(available_controls)

        # Manually prepare data to handle missing values before fitting
        all_vars_in_formula = [dependent_var, ai_factor, sector_col, 'Year', cluster_col] + available_controls
        # Filter for columns that actually exist in the dataframe
        existing_vars = [v for v in all_vars_in_formula if v is not None and v in reg_data.columns]
        
        clean_data = reg_data[existing_vars].dropna()

        if len(clean_data) < 20: # Check for sufficient data
            print(f"Skipping model '{model_name}' due to insufficient data ({len(clean_data)} obs).")
            results[model_name] = None
            continue

        try:
            # Run the model on clean data with firm-clustered standard errors
            model = smf.ols(formula, data=clean_data).fit(
                cov_type='cluster', 
                cov_kwds={'groups': clean_data[cluster_col]}
            )
            
            # Store results for the AI factor
            ai_coef = model.params.get(ai_factor, np.nan)
            ai_stderr = model.bse.get(ai_factor, np.nan)
            ai_tstat = model.tvalues.get(ai_factor, np.nan)
            ai_pvalue = model.pvalues.get(ai_factor, np.nan)
            
            # Store results for control variables
            control_stats = {}
            for control in available_controls:
                control_stats[control] = {
                    'coef': model.params.get(control, np.nan),
                    'stderr': model.bse.get(control, np.nan),
                    'tstat': model.tvalues.get(control, np.nan),
                    'pvalue': model.pvalues.get(control, np.nan)
                }
            
            results[model_name] = {
                'model': model, 'ai_coefficient': ai_coef, 'ai_stderr': ai_stderr,
                'ai_tstat': ai_tstat, 'ai_pvalue': ai_pvalue, 'control_stats': control_stats,
                'r_squared': model.rsquared, 'adj_r_squared': model.rsquared_adj,
                'n_obs': model.nobs, 'available_controls': available_controls
            }
        except Exception as e:
            print(f"Error running regression for model '{model_name}'. Formula: '{formula}'. Error: {e}")
            results[model_name] = None
    
    return results

def format_coefficient_academic(coef, stderr, pvalue):
    """
    Formats a coefficient, its standard error, and adds significance stars.
    """
    if pd.isna(coef) or pd.isna(stderr):
        return "", ""
    
    if pvalue < 0.01: stars = "***"
    elif pvalue < 0.05: stars = "**"
    elif pvalue < 0.1: stars = "*"
    else: stars = ""
    
    coef_str = f"{coef:.4f}{stars}"
    stderr_str = f"({stderr:.4f})"
    
    return coef_str, stderr_str

def create_book_controls_table_single_factor(reg_data, ai_factor, dependent_var, sector_col='Sector', cluster_col='Ticker'):
    """
    Assembles a full academic-style regression table for one AI factor with book controls.
    """
    results = run_book_controls_regression_table(reg_data, ai_factor, dependent_var, sector_col, cluster_col)
    
    if results is None:
        return pd.DataFrame(), None

    # Define the order and display names of variables in the table
    variable_order = [
        ai_factor, 'calc_log_market_cap', 'calc_price_to_book', 'calc_roa'
    ]
    variable_names = {
        ai_factor: ai_factor.replace('_', ' ').title(),
        'calc_log_market_cap': 'Log(Market Cap)',
        'calc_price_to_book': 'Price-to-Book',
        'calc_roa': 'Return on Assets (ROA)'
    }
    
    model_keys = ['Base', 'Base + Size', 'Base + Size + Value', 'Full Controls']
    
    table_data = []
    for var in variable_order:
        row_data = {'Variable': variable_names.get(var, var)}
        for model_name in model_keys:
            if results.get(model_name) is None:
                row_data[f'{model_name}_coef'], row_data[f'{model_name}_stderr'] = "", ""
                continue
                
            if var == ai_factor:
                coef, stderr, pvalue = results[model_name]['ai_coefficient'], results[model_name]['ai_stderr'], results[model_name]['ai_pvalue']
            else:
                control_stats = results[model_name]['control_stats']
                if var in control_stats:
                    coef, stderr, pvalue = control_stats[var]['coef'], control_stats[var]['stderr'], control_stats[var]['pvalue']
                else:
                    coef = stderr = pvalue = np.nan
            
            coef_str, stderr_str = format_coefficient_academic(coef, stderr, pvalue)
            row_data[f'{model_name}_coef'] = coef_str
            row_data[f'{model_name}_stderr'] = stderr_str
            
        table_data.append(row_data)
    
    # Add summary statistics (R-squared, N Obs, etc.)
    summary_rows = []
    for stat_name in ['R²', 'Adj. R²', 'Observations']:
        row = {'Variable': stat_name}
        for model_name in model_keys:
            if results.get(model_name) is not None:
                if stat_name == 'R²': value = f"{results[model_name]['r_squared']:.4f}"
                elif stat_name == 'Adj. R²': value = f"{results[model_name]['adj_r_squared']:.4f}"
                else: value = f"{int(results[model_name]['n_obs']):,}"
                row[f'{model_name}_coef'] = value
                row[f'{model_name}_stderr'] = ""
            else:
                row[f'{model_name}_coef'], row[f'{model_name}_stderr'] = "", ""
        summary_rows.append(row)
        
    return pd.DataFrame(table_data + summary_rows), results

def export_book_controls_tables(all_tables):
    """
    Exports book control regression results into a detailed summary and individual tables.
    """
    print(f"\nExporting book control regression results")
    print("=" * 45)
    
    # 1. Export a detailed summary CSV
    summary_data = []
    for ai_factor, factor_tables in all_tables.items():
        for horizon, table_info in factor_tables.items():
            for model_name, result in table_info['results'].items():
                if result:
                    summary_data.append({
                        'AI_Factor': ai_factor, 'Horizon': horizon,
                        'Dependent_Variable': table_info['dependent_var'], 'Model': model_name,
                        'AI_Coefficient': result['ai_coefficient'], 'AI_Std_Error': result['ai_stderr'],
                        'AI_t_stat': result['ai_tstat'], 'AI_p_value': result['ai_pvalue'],
                        'R_Squared': result['r_squared'], 'Adj_R_Squared': result['adj_r_squared'],
                        'N_Obs': result['n_obs']
                    })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('book_controls_regression_summary_CLUSTERED.csv', index=False)
    print(f"Detailed summary exported to: book_controls_regression_summary_CLUSTERED.csv")

    # 2. Export individual, publication-ready tables
    print("\n2. Exporting individual, publication-ready tables...")
    exported_count = 0
    model_keys = ['Base', 'Base + Size', 'Base + Size + Value', 'Full Controls']
    model_headers = ['Base Model', '+ Size', '+ Value', '+ ROA']

    for ai_factor, factor_tables in all_tables.items():
        for horizon, table_info in factor_tables.items():
            table_df = table_info['table_df']
            
            output_rows = [['Variable'] + model_headers]
            for _, row in table_df.iterrows():
                # Coefficient row
                output_rows.append([row['Variable']] + [row[f'{m}_coef'] for m in model_keys])
                # Standard error row (if it contains data)
                if any(row[f'{m}_stderr'] for m in model_keys):
                    output_rows.append([''] + [row[f'{m}_stderr'] for m in model_keys])

            export_df = pd.DataFrame(output_rows[1:], columns=output_rows[0])
            export_df.to_csv(f"table_book_controls_{ai_factor}_{horizon}_CLUSTERED.csv", index=False)
            exported_count += 1
            
    print(f"All {exported_count} individual tables exported (e.g., 'table_book_controls_AI_Cumulative_Score_t12_CLUSTERED.csv').")


def display_sample_book_controls_table(all_tables, ai_factor='AI_Cumulative_Score', horizon='t12'):
    """
    Displays a single, formatted sample table in the console for a quick check.
    """
    if ai_factor in all_tables and horizon in all_tables[ai_factor]:
        table_info = all_tables[ai_factor][horizon]
        table_df = table_info['table_df']
        
        print(f"\n\nSample table (book controls buildup with firm-clustered SE):")
        print(f"AI Factor: {ai_factor.replace('_', ' ').title()} ({horizon}) | Dependent Var: {table_info['dependent_var']}")
        print("=" * 90)
        
        model_keys = ['Base', 'Base + Size', 'Base + Size + Value', 'Full Controls']
        headers = ['Base Model', '+ Size', '+ Value', '+ ROA']
        print(f"{'Variable':<25} {'':<15} {'':<15} {'':<15} {'':<15}".format(*headers))
        print(f"{'':<25} {headers[0]:<15} {headers[1]:<15} {headers[2]:<15} {headers[3]:<15}")
        print("-" * 90)
        
        for _, row in table_df.iterrows():
            var_name = row['Variable'][:24]
            print(f"{var_name:<25} {row[f'{model_keys[0]}_coef']:<15} {row[f'{model_keys[1]}_coef']:<15} {row[f'{model_keys[2]}_coef']:<15} {row[f'{model_keys[3]}_coef']:<15}")
            # Print standard error row if it's not empty
            if any(row[f'{m}_stderr'] for m in model_keys):
                print(f"{'':<25} {row[f'{model_keys[0]}_stderr']:<15} {row[f'{model_keys[1]}_stderr']:<15} {row[f'{model_keys[2]}_stderr']:<15} {row[f'{model_keys[3]}_stderr']:<15}")
        
        print("\nNotes: Firm-clustered standard errors (by Ticker) in parentheses. *p<0.1, **p<0.05, ***p<0.01.")


def run_all_book_controls_tables(reg_data, ai_factors, export_tables=True, cluster_col='Ticker'):
    """
    Main orchestrator function. Runs regression analysis for book controls and exports results.
    """
    
    print("Creating book control regression tables with firm-clustered standard errors")
    print("=" * 75)
    print(f"Clustering standard errors by firm identifier: '{cluster_col}'")
    
    # Also test the cumulative score
    all_ai_factors_to_test = ai_factors + ['AI_Cumulative_Score']
    
    # Define return horizons and corresponding dependent variable names
    horizons = {
        't3': 'excess_return_3mo', 't6': 'excess_return_6mo',
        't9': 'excess_return_9mo', 't12': 'excess_return_12mo'
    }
    
    all_tables = {}
    
    for ai_factor in all_ai_factors_to_test:
        if ai_factor not in reg_data.columns:
            print(f"'{ai_factor}' not found in data, skipping...")
            continue
            
        print(f"\nCreating tables for: {ai_factor.replace('_', ' ').title()}")
        factor_tables = {}
        
        for horizon, dependent_var in horizons.items():
            if dependent_var not in reg_data.columns:
                print(f"  Dependent var '{dependent_var}' not found, skipping {horizon}")
                continue
                
            print(f"  Running for {horizon} horizon ({dependent_var})")
            
            # This is the core function call to generate the table data for one factor/horizon pair
            table_df, results = create_book_controls_table_single_factor(
                reg_data, ai_factor, dependent_var, cluster_col=cluster_col
            )
            
            if results: 
                factor_tables[horizon] = {
                    'table_df': table_df, 'results': results, 'dependent_var': dependent_var
                }
            
        all_tables[ai_factor] = factor_tables
    
    if export_tables:
        export_book_controls_tables(all_tables)
    
    return all_tables, reg_data

# example execution

print("\n\nBook controls regression table generator (firm-clustered SE version)")
print("=" * 75)
print("This script takes the master 'final_reg_data' and runs all regressions.")
print("It builds up models by progressively adding book control variables.")
print("It uses firm-clustered standard errors for valid inference in panel data.")

# 1. Define the list of individual, averaged AI factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# 2. Run the complete analysis using the master dataframe ('final_reg_data')
all_book_controls_tables, final_data_for_reg = run_all_book_controls_tables(
    final_reg_data, 
    ai_factors, 
    cluster_col='Ticker'
)

# 3. Display a sample table to check the results for the main cumulative score
display_sample_book_controls_table(
    all_book_controls_tables, 
    ai_factor='AI_Cumulative_Score', 
    horizon='t12'
)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# function definitions

def run_single_regression(reg_data, formula, cluster_col='Ticker'):
    """
    helper function: Runs a single regression with robust error handling for missing data.
    """
    all_vars_in_formula = [v.strip() for v in formula.replace('~', '+').replace('*', '+').split('+')]
    cleaned_vars = []
    for var in all_vars_in_formula:
        if 'C(' in var:
            cleaned_vars.append(var[var.find('(')+1:var.find(')')])
        else:
            cleaned_vars.append(var)
    
    final_vars_set = set(cleaned_vars + [cluster_col])
    final_vars = [v for v in final_vars_set if v in reg_data.columns]
    
    clean_data = reg_data[final_vars].dropna()

    if len(clean_data) < 20:
        print(f"Skipping model due to insufficient data ({len(clean_data)} obs). Formula: {formula}")
        return None

    try:
        model = smf.ols(formula, data=clean_data).fit(
            cov_type='cluster', 
            cov_kwds={'groups': clean_data[cluster_col]}
        )
        return model
    except Exception as e:
        print(f"Error running regression. Formula: '{formula}'. Error: {e}")
        return None

def run_full_control_regression(reg_data, ai_factor, dependent_var, sector_col='Sector', cluster_col='Ticker'):
    """
    Run full control variables regression for a single AI factor using firm-clustered SEs.
    """
    control_vars = ['calc_log_market_cap', 'calc_price_to_book', 'calc_roa']
    available_controls = [var for var in control_vars if var in reg_data.columns]
    
    formula = f"{dependent_var} ~ {ai_factor}"
    if sector_col and sector_col in reg_data.columns:
        formula += f" + C({sector_col})"
    if 'Year' in reg_data.columns:
        formula += f" + C(Year)"
    if available_controls:
        formula += " + " + " + ".join(available_controls)
    
    model = run_single_regression(reg_data, formula, cluster_col)
    
    if model is None:
        return None
        
    return {
        'model': model,
        'ai_coefficient': model.params.get(ai_factor, np.nan),
        'ai_stderr': model.bse.get(ai_factor, np.nan),
        'ai_pvalue': model.pvalues.get(ai_factor, np.nan),
        'r_squared': model.rsquared, 'adj_r_squared': model.rsquared_adj,
        'n_obs': model.nobs
    }

def run_full_ff6_regression(reg_data, ai_factor, dependent_var, beta_horizon='t12', sector_col='Sector', cluster_col='Ticker'):
    """
    Run full FF6 regression for a single AI factor using firm-clustered SEs.
    """
    ff6_factors = [f'beta_mktrf_{beta_horizon}', f'beta_smb_{beta_horizon}', f'beta_hml_{beta_horizon}',
                   f'beta_rmw_{beta_horizon}', f'beta_cma_{beta_horizon}', f'beta_umd_{beta_horizon}']
    available_ff_factors = [var for var in ff6_factors if var in reg_data.columns]
    
    formula = f"{dependent_var} ~ {ai_factor}"
    if sector_col and sector_col in reg_data.columns:
        formula += f" + C({sector_col})"
    if 'Year' in reg_data.columns:
        formula += f" + C(Year)"
    if available_ff_factors:
        formula += " + " + " + ".join(available_ff_factors)
        
    model = run_single_regression(reg_data, formula, cluster_col)
    
    if model is None:
        return None

    return {
        'model': model,
        'ai_coefficient': model.params.get(ai_factor, np.nan),
        'ai_stderr': model.bse.get(ai_factor, np.nan),
        'ai_pvalue': model.pvalues.get(ai_factor, np.nan),
        'r_squared': model.rsquared, 'adj_r_squared': model.rsquared_adj,
        'n_obs': model.nobs
    }

def format_coefficient_for_comparison_table(coef, stderr, pvalue):
    """
    Format coefficient for comparison table (coefficient with stars and stderr in parentheses).
    """
    if pd.isna(coef) or pd.isna(stderr):
        return ""
    
    if pvalue < 0.01: stars = "***"
    elif pvalue < 0.05: stars = "**"
    elif pvalue < 0.1: stars = "*"
    else: stars = ""
    
    return f"{coef:.4f}{stars}\n({stderr:.4f})"

def create_ai_factors_comparison_table(reg_data, ai_factors, cluster_col='Ticker', export_tables=True):
    """
    Create cleaner comparison tables without extra empty rows.
    """
    print("Creating AI factors comparison tables (controls vs. FF6)")
    print("=" * 65)
    
    horizons = {'t3': 'excess_return_3mo', 't6': 'excess_return_6mo', 't9': 'excess_return_9mo', 't12': 'excess_return_12mo'}
    all_stacked_rows = [["AI FACTOR COMPARISON: CONTROLS vs. FAMA-FRENCH 6-FACTOR", "", ""]]
    
    for horizon, dependent_var in horizons.items():
        if dependent_var not in reg_data.columns:
            print(f"Skipping {horizon} horizon, dependent variable '{dependent_var}' not found.")
            continue
            
        print(f"\nProcessing {horizon.upper()} Horizon (Dependent Var: {dependent_var})")
        
        horizon_results = {}
        all_ai_to_test = ai_factors + ['AI_Cumulative_Score']
        for ai_factor in all_ai_to_test:
            if ai_factor in reg_data.columns:
                control_res = run_full_control_regression(reg_data, ai_factor, dependent_var, cluster_col=cluster_col)
                ff6_res = run_full_ff6_regression(reg_data, ai_factor, dependent_var, horizon, cluster_col=cluster_col)
                horizon_results[ai_factor] = {'control_result': control_res, 'ff6_result': ff6_res}

        # Format the table for this horizon
        table_rows = [[f"Time Horizon: {horizon.upper()}", "", ""], [f"Standard Errors: Clustered by {cluster_col}", "", ""],
                      ["AI Factor", "Control Variables Model", "Fama-French 6-Factor Model"],
                      ["", "(FE + Size, Value, ROA)", "(FE + FF6 Factors)"]]

        for ai_factor, results in horizon_results.items():
            control_formatted = format_coefficient_for_comparison_table(results['control_result']['ai_coefficient'], results['control_result']['ai_stderr'], results['control_result']['ai_pvalue']) if results.get('control_result') else "Error"
            ff6_formatted = format_coefficient_for_comparison_table(results['ff6_result']['ai_coefficient'], results['ff6_result']['ai_stderr'], results['ff6_result']['ai_pvalue']) if results.get('ff6_result') else "Error"
            table_rows.append([ai_factor.replace('_', ' ').title(), control_formatted, ff6_formatted])
        
        # Add summary rows for R-squared and Observations without extra spacing
        control_r2s = [res['control_result']['adj_r_squared'] for res in horizon_results.values() if res.get('control_result')]
        ff6_r2s = [res['ff6_result']['adj_r_squared'] for res in horizon_results.values() if res.get('ff6_result')]
        avg_control_r2 = np.mean(control_r2s) if control_r2s else np.nan
        avg_ff6_r2 = np.mean(ff6_r2s) if ff6_r2s else np.nan
        table_rows.append(["Adj. R-squared (Avg)", f"{avg_control_r2:.4f}", f"{avg_ff6_r2:.4f}"])

        control_n = [res['control_result']['n_obs'] for res in horizon_results.values() if res.get('control_result')]
        ff6_n = [res['ff6_result']['n_obs'] for res in horizon_results.values() if res.get('ff6_result')]
        avg_control_n = np.mean(control_n) if control_n else np.nan
        avg_ff6_n = np.mean(ff6_n) if ff6_n else np.nan
        table_rows.append(["Observations (Avg)", f"{avg_control_n:,.0f}", f"{avg_ff6_n:,.0f}"])
            
        all_stacked_rows.extend(table_rows)

    if export_tables:
        filename = f"ai_factors_comparison_controls_vs_ff6_CLUSTERED.csv"
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            import csv
            csv.writer(f).writerows(all_stacked_rows)
        print(f"\n\nFull comparison table exported to: {filename}")

    return all_stacked_rows

# run the analysis

print("AI factors comparison: controls vs Fama-French (firm-clustered SE version)")
print("=" * 75)
print("This script runs regressions comparing a simple control model to a full FF6 model.")
print("It uses firm-clustered standard errors for valid inference in panel data.")
print("It now includes Adj. R-squared and N observations in the final table.")

# 1. Define the list of individual, averaged AI factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# 2. Run the complete analysis using the master dataframe ('final_reg_data')
comparison_table_rows = create_ai_factors_comparison_table(
    reg_data=final_reg_data, 
    ai_factors=ai_factors, 
    cluster_col='Ticker'
)

print("\n\nAI Factor Comparison Analysis Complete!")

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

#================================================================================
#                        1. HELPER FUNCTION
#================================================================================

def run_single_regression(reg_data, formula, cluster_col='Ticker'):
    """
    **HELPER FUNCTION**: Runs a single regression with robust error handling for missing data
    and uses firm-clustered standard errors.
    """
    # Identify all variables needed for this specific formula
    all_vars_in_formula = [v.strip() for v in formula.replace('~', '+').replace('*', '+').replace(':', '+').split('+')]
    cleaned_vars = []
    for var in all_vars_in_formula:
        if 'C(' in var:
            cleaned_vars.append(var[var.find('(')+1:var.find(')')])
        else:
            cleaned_vars.append(var)
    
    final_vars_set = set(cleaned_vars + [cluster_col])
    final_vars = [v for v in final_vars_set if v in reg_data.columns]
    
    clean_data = reg_data[final_vars].dropna()

    if len(clean_data) < 20:
        # print(f"Skipping model due to insufficient data ({len(clean_data)} obs).")
        return None

    try:
        model = smf.ols(formula, data=clean_data).fit(
            cov_type='cluster', 
            cov_kwds={'groups': clean_data[cluster_col]}
        )
        return model
    except Exception as e:
        print(f"Error in formula '{formula}': {e}")
        return None

#================================================================================
#                        2. MAIN ANALYSIS FUNCTIONS
#================================================================================

def run_sector_interactions(data, ai_factor, dependent_var, ff_factors, controls, sectors, cluster_col):
    """
    Run sector interaction models for a single AI factor.
    """
    print(f"\n  INTERACTION MODEL for: {ai_factor}")
    factor_models = {}
    
    # Model: Combined (Controls + FF6 + Interactions)
    control_terms = " + ".join(controls)
    ff_terms = " + ".join(ff_factors)
    formula = f"{dependent_var} ~ {ai_factor}*C(Sector) + {control_terms} + {ff_terms} + C(Year)"
    
    model = run_single_regression(data, formula, cluster_col)
    
    if model:
        factor_models['combined'] = model
        factor_models['sector_effects'] = extract_sector_effects(model, ai_factor, sectors)
        print(f"    Combined Interaction Model: R²={model.rsquared:.3f}, N={model.nobs:,}")
    else:
        print(f"    Combined Interaction Model failed.")
        
    return factor_models

def run_individual_sectors(data, ai_factor, dependent_var, ff_factors, controls, sectors, cluster_col):
    """
    Run regressions for each sector individually.
    """
    print(f"\n  INDIVIDUAL SECTOR REGRESSIONS for: {ai_factor}")
    results = {}
    
    for sector in sectors:
        sector_data = data[data['Sector'] == sector]
        if len(sector_data) < 20: continue

        # Combined Model: AI Factor + Controls + FF6 + Year FE
        formula = f"{dependent_var} ~ {ai_factor} + {' + '.join(controls)} + {' + '.join(ff_factors)} + C(Year)"
        model = run_single_regression(sector_data, formula, cluster_col)
        
        if model:
            if sector not in results: results[sector] = {}
            results[sector][ai_factor] = model
            
    return results

def extract_sector_effects(model, ai_factor, sectors):
    """
    Extract sector-specific AI effects from an interaction model.
    """
    effects = {}
    # Base effect (for the reference sector)
    base_coef = model.params.get(ai_factor, 0)
    base_se = model.bse.get(ai_factor, np.nan)
    base_pval = model.pvalues.get(ai_factor, np.nan)
    
    # The reference sector is the first one alphabetically that patsy chooses
    ref_sector = [s for s in sectors if f"C(Sector)[T.{s}]" not in model.params.index][0]
    effects[ref_sector] = {'coef': base_coef, 'se': base_se, 'pvalue': base_pval}

    # Interaction effects for other sectors
    for sector in sectors:
        if sector == ref_sector: continue
        interaction_term = f"{ai_factor}:C(Sector)[T.{sector}]"
        if interaction_term in model.params:
            # The total effect for an interacted sector is (base_coef + interaction_coef)
            # We test the significance of the total effect using a Wald test.
            total_effect_test = model.t_test(f"{ai_factor} + {interaction_term}")
            effects[sector] = {
                'coef': total_effect_test.effect[0],
                'se': total_effect_test.sd[0],
                'pvalue': total_effect_test.pvalue
            }
    return effects

#================================================================================
#                        3. EXPORT & FORMATTING FUNCTIONS
#================================================================================

def export_sector_analysis_results(all_results, ai_factors, sectors, cluster_col):
    """
    **NEW**: Creates a single, comprehensive CSV file summarizing all sector results.
    """
    print("\n\n EXPORTING FINAL SECTOR ANALYSIS")
    print("=" * 40)

    output_rows = []
    header = ['Time Horizon', 'Analysis Type', 'Sector/Model', 'AI Factor', 'Coefficient', 'Std. Error', 'P-Value', 'Significant (5%)', 'N Obs', 'Adj. R-squared']
    output_rows.append(header)

    for horizon, horizon_results in all_results.items():
        # Interaction Model Results
        for ai_factor in ai_factors:
            if 'interactions' in horizon_results and ai_factor in horizon_results['interactions'] and 'sector_effects' in horizon_results['interactions'][ai_factor]:
                model = horizon_results['interactions'][ai_factor].get('combined')
                if not model: continue
                
                for sector, effects in horizon_results['interactions'][ai_factor]['sector_effects'].items():
                    output_rows.append([
                        horizon.upper(), 'Interaction', sector, ai_factor.replace('_', ' ').title(),
                        f"{effects.get('coef', 0):.4f}", f"{effects.get('se', 0):.4f}", f"{effects.get('pvalue', 1):.4f}",
                        effects.get('pvalue', 1) < 0.05, model.nobs, f"{model.rsquared_adj:.4f}"
                    ])

        # Individual Sector Regression Results
        if 'individual' in horizon_results:
            for sector, sector_results in horizon_results['individual'].items():
                for ai_factor, model in sector_results.items():
                    output_rows.append([
                        horizon.upper(), 'Individual', sector, ai_factor.replace('_', ' ').title(),
                        f"{model.params.get(ai_factor, 0):.4f}", f"{model.bse.get(ai_factor, 0):.4f}", f"{model.pvalues.get(ai_factor, 1):.4f}",
                        model.pvalues.get(ai_factor, 1) < 0.05, model.nobs, f"{model.rsquared_adj:.4f}"
                    ])
    
    filename = f"sector_analysis_summary_CLUSTERED.csv"
    pd.DataFrame(output_rows[1:], columns=output_rows[0]).to_csv(filename, index=False)
    print(f" Comprehensive sector analysis summary exported to: {filename}")

#================================================================================
#                        4. MAIN ORCHESTRATOR
#================================================================================

def run_final_sector_analysis(final_reg_data, ai_factors, time_horizons=['t12'], cluster_col='Ticker'):
    """
    Complete sector analysis using final_reg_data with firm-clustered SEs.
    """
    print(" FINAL SECTOR ANALYSIS (FIRM-CLUSTERED SEs)")
    print("=" * 50)
    
    dependent_vars = {'t3': 'excess_return_3mo', 't6': 'excess_return_6mo', 't9': 'excess_return_9mo', 't12': 'excess_return_12mo'}
    sectors = sorted([s for s in final_reg_data['Sector'].unique() if pd.notna(s)])
    controls = ['calc_log_market_cap', 'calc_price_to_book', 'calc_roa']
    all_ai_factors = ai_factors + ['AI_Cumulative_Score']

    all_results = {}
    for horizon in time_horizons:
        if dependent_vars[horizon] not in final_reg_data.columns:
            print(f"\n⚠ Skipping {horizon} horizon: Dependent variable not available.")
            continue
            
        print(f"\n ANALYZING {horizon.upper()} HORIZON")
        ff_factors = [f'beta_mktrf_{horizon}', f'beta_smb_{horizon}', f'beta_hml_{horizon}',
                      f'beta_rmw_{horizon}', f'beta_cma_{horizon}', f'beta_umd_{horizon}']
        
        #  only need to run one loop for each AI factor now
        interaction_results_all = {}
        individual_results_all = {}

        for ai_factor in all_ai_factors:
            interaction_results_all.update(run_sector_interactions(final_reg_data, ai_factor, dependent_vars[horizon], ff_factors, controls, sectors, cluster_col))
            individual_results_all.update(run_individual_sectors(final_reg_data, ai_factor, dependent_vars[horizon], ff_factors, controls, sectors, cluster_col))

        all_results[horizon] = {
            'interactions': interaction_results_all,
            'individual': individual_results_all
        }
        
    export_sector_analysis_results(all_results, all_ai_factors, sectors, cluster_col)
    
    print(f"\n ANALYSIS COMPLETE!")
    return all_results

#================================================================================
#                        5. RUN THE ANALYSIS
#================================================================================

ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# Run for the 12-month horizon as a sample
sector_analysis_results = run_final_sector_analysis(final_reg_data, ai_factors, time_horizons=['t12'])

🚀 FINAL SECTOR ANALYSIS (FIRM-CLUSTERED SEs)

🚀 ANALYZING T12 HORIZON

  🔍 INTERACTION MODEL for: AI_Washing
    ✅ Combined Interaction Model: R²=0.052, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: AI_Washing

  🔍 INTERACTION MODEL for: Disclosure_Sentiment
    ✅ Combined Interaction Model: R²=0.050, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: Disclosure_Sentiment

  🔍 INTERACTION MODEL for: Forward_Looking
    ✅ Combined Interaction Model: R²=0.052, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: Forward_Looking

  🔍 INTERACTION MODEL for: Strategic_Depth
    ✅ Combined Interaction Model: R²=0.055, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: Strategic_Depth

  🔍 INTERACTION MODEL for: Talent_Investment
    ✅ Combined Interaction Model: R²=0.051, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: Talent_Investment

  🔍 INTERACTION MODEL for: Risk_External
    ✅ Combined Interaction Model: R²=0.041, N=3,028.0

  🏭 INDIVIDUAL SECTOR REGRESSIONS for: Risk_External

  🔍 I

In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

#================================================================================
#                        1. HELPER & REGRESSION FUNCTIONS
#================================================================================

def run_single_regression(reg_data, formula, cluster_col='Ticker'):
    """
    **HELPER FUNCTION**: Runs a single regression with robust error handling for missing data
    and uses firm-clustered standard errors.
    """
    # Identify all variables needed for this specific formula
    all_vars_in_formula = [v.strip() for v in formula.replace('~', '+').replace('*', '+').replace(':', '+').split('+')]
    cleaned_vars = []
    for var in all_vars_in_formula:
        if 'C(' in var:
            cleaned_vars.append(var[var.find('(')+1:var.find(')')])
        else:
            cleaned_vars.append(var)
    
    final_vars_set = set(cleaned_vars + [cluster_col])
    final_vars = [v for v in final_vars_set if v in reg_data.columns]
    
    clean_data = reg_data[final_vars].dropna()

    if len(clean_data) < 20:
        return None

    try:
        model = smf.ols(formula, data=clean_data).fit(
            cov_type='cluster', 
            cov_kwds={'groups': clean_data[cluster_col]}
        )
        return model
    except Exception as e:
        # This will now only print an error if one actually occurs
        # print(f"Error in formula '{formula}': {e}")
        return None

def format_coefficient_academic(coef, stderr, pvalue):
    """
    **FIXED**: Formats a coefficient, SE, and p-value into a standard academic string
    with standard errors in parentheses on a new line.
    """
    if pd.isna(coef) or pd.isna(stderr):
        return ""
    if pvalue < 0.01: stars = "***"
    elif pvalue < 0.05: stars = "**"
    elif pvalue < 0.1: stars = "*"
    else: stars = ""
    
    # This format creates "COEF***\n(SE)" which displays well in Excel/Sheets
    return f"{coef:.4f}{stars}\n({stderr:.4f})"

#================================================================================
#                        2. SECTOR ANALYSIS FUNCTIONS
#================================================================================

def run_individual_sector_regressions(data, ai_factors, dependent_var, ff_factors, controls, sectors, cluster_col):
    """
    **MODIFIED**: Runs regressions for each AI factor within each individual sector,
    but only for the 'Controls' and 'FF6' models (removes 'Combined').
    """
    print("   Running Individual Sector Regressions (Controls vs. FF6)...")
    sector_results = {}
    
    for sector in sectors:
        sector_data = data[data['Sector'] == sector]
        if len(sector_data) < 20: continue

        models_for_this_sector = {}
        for ai_factor in ai_factors:
            if ai_factor not in sector_data.columns: continue

            # Define the two models to run for each AI factor
            control_formula = f"{dependent_var} ~ {ai_factor} + {' + '.join(controls)} + C(Year)"
            ff6_formula = f"{dependent_var} ~ {ai_factor} + {' + '.join(ff_factors)} + C(Year)"
            
            models_for_this_sector[ai_factor] = {
                'Controls': run_single_regression(sector_data, control_formula, cluster_col),
                'FF6': run_single_regression(sector_data, ff6_formula, cluster_col)
            }
        sector_results[sector] = models_for_this_sector
        
    return sector_results

#================================================================================
#                        3. EXPORT & FORMATTING FUNCTIONS
#================================================================================

def create_and_export_sector_tables(sector_results, horizon, cluster_col):
    """
    **MODIFIED**: Creates and exports publication-ready academic tables for each sector,
    with a cleaner format and added summary statistics.
    """
    output_dir = f"sector_analysis/{horizon}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n   Exporting individual sector tables to: '{output_dir}/'")

    for sector, ai_factor_models in sector_results.items():
        table_rows = []
        # **FIX**: Simplified header
        header = ["AI Factor", "Controls Model", "Fama-French 6-Factor Model"]
        table_rows.append(header)

        # Temp storage for R-squared and N values to average them later
        r2_controls, r2_ff6 = [], []
        n_controls, n_ff6 = [], []

        for ai_factor, models in ai_factor_models.items():
            row_data = [ai_factor.replace('_', ' ').title()]
            
            # Process 'Controls' model
            model_controls = models.get('Controls')
            if model_controls:
                coef, se, pval = model_controls.params.get(ai_factor), model_controls.bse.get(ai_factor), model_controls.pvalues.get(ai_factor)
                row_data.append(format_coefficient_academic(coef, se, pval))
                r2_controls.append(model_controls.rsquared_adj)
                n_controls.append(model_controls.nobs)
            else:
                row_data.append("N/A")

            # Process 'FF6' model
            model_ff6 = models.get('FF6')
            if model_ff6:
                coef, se, pval = model_ff6.params.get(ai_factor), model_ff6.bse.get(ai_factor), model_ff6.pvalues.get(ai_factor)
                row_data.append(format_coefficient_academic(coef, se, pval))
                r2_ff6.append(model_ff6.rsquared_adj)
                n_ff6.append(model_ff6.nobs)
            else:
                row_data.append("N/A")
                
            table_rows.append(row_data)
            
        # **FIX**: Add summary stats (Avg R-squared, Avg N) directly to the table footer
        table_rows.append(["Adj. R-squared (Avg)", f"{np.mean(r2_controls):.4f}" if r2_controls else "N/A", f"{np.mean(r2_ff6):.4f}" if r2_ff6 else "N/A"])
        table_rows.append(["Observations (Avg)", f"{np.mean(n_controls):,.0f}" if n_controls else "N/A", f"{np.mean(n_ff6):,.0f}" if n_ff6 else "N/A"])
        
        # Create and save the DataFrame
        table_df = pd.DataFrame(table_rows[1:], columns=table_rows[0])
        filename = f"{output_dir}/{sector.replace(' ', '_').replace('/', '_')}.csv"
        table_df.to_csv(filename, index=False)

    print(f"     Successfully exported {len(sector_results)} sector tables.")

#================================================================================
#                        4. MAIN ORCHESTRATOR
#================================================================================

def run_final_sector_analysis(final_reg_data, ai_factors, time_horizons=['t12'], cluster_col='Ticker'):
    """
    **MODIFIED**: Main function to run the simplified (Controls vs. FF6) sector analysis.
    """
    print("🚀 FINAL SECTOR ANALYSIS (FIRM-CLUSTERED SEs & INDIVIDUAL TABLE EXPORTS)")
    print("=" * 75)
    
    dependent_vars = {'t3': 'excess_return_3mo', 't6': 'excess_return_6mo', 't9': 'excess_return_9mo', 't12': 'excess_return_12mo'}
    sectors = sorted([s for s in final_reg_data['Sector'].unique() if pd.notna(s)])
    controls = ['calc_log_market_cap', 'calc_price_to_book', 'calc_roa']
    all_ai_factors = ai_factors + ['AI_Cumulative_Score']

    for horizon in time_horizons:
        if dependent_vars.get(horizon) not in final_reg_data.columns:
            print(f"\n⚠️ Skipping {horizon.upper()} horizon: Dependent variable not available.")
            continue
            
        print(f"\n🚀 ANALYZING {horizon.upper()} HORIZON...")
        
        ff_factors = [f'beta_mktrf_{horizon}', f'beta_smb_{horizon}', f'beta_hml_{horizon}',
                      f'beta_rmw_{horizon}', f'beta_cma_{horizon}', f'beta_umd_{horizon}']
        
        individual_sector_results = run_individual_sector_regressions(
            final_reg_data, all_ai_factors, dependent_vars[horizon], 
            ff_factors, controls, sectors, cluster_col
        )
        
        create_and_export_sector_tables(individual_sector_results, horizon, cluster_col)
        
    print("\n\n All Sector Analysis and Exports Complete")

#================================================================================
#                        5. RUN THE ANALYSIS
#================================================================================

# Define the list of your individual, averaged AI factors
ai_factors = [
    'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
    'Strategic_Depth', 'Talent_Investment', 'Risk_External',
    'Risk_Non_Adoption', 'Risk_Own_Adoption'
]

# Run the full analysis for all time horizons
# This will create the 'sector_analysis' folder with subfolders for each horizon.
run_final_sector_analysis(final_reg_data, ai_factors, time_horizons=['t3', 't6', 't9', 't12'])


🚀 FINAL SECTOR ANALYSIS (FIRM-CLUSTERED SEs & INDIVIDUAL TABLE EXPORTS)

🚀 ANALYZING T3 HORIZON...
  📊 Running Individual Sector Regressions (Controls vs. FF6)...

  📁 Exporting individual sector tables to: 'sector_analysis/t3/'
    ✅ Successfully exported 11 sector tables.

🚀 ANALYZING T6 HORIZON...
  📊 Running Individual Sector Regressions (Controls vs. FF6)...

  📁 Exporting individual sector tables to: 'sector_analysis/t6/'
    ✅ Successfully exported 11 sector tables.

🚀 ANALYZING T9 HORIZON...
  📊 Running Individual Sector Regressions (Controls vs. FF6)...

  📁 Exporting individual sector tables to: 'sector_analysis/t9/'
    ✅ Successfully exported 11 sector tables.

🚀 ANALYZING T12 HORIZON...
  📊 Running Individual Sector Regressions (Controls vs. FF6)...

  📁 Exporting individual sector tables to: 'sector_analysis/t12/'
    ✅ Successfully exported 11 sector tables.


✅✅✅ All Sector Analysis and Exports Complete! ✅✅✅


In [ ]:
import pandas as pd
import numpy as np

def find_interesting_cases(reg_data, sector_name='Consumer Discretionary', return_col='excess_return_12mo'):
    """
    identifies and analyzes firms within a specific sector that have high ai scores
    but subsequently underperform, providing concrete examples for case studies
    """
    print("Case study explorer")
    print("=" * 50)
    print(f"Objective: find firms in '{sector_name}' with high AI scores and low subsequent returns")
    
    # 1. filter the dataset to the specified sector
    sector_df = reg_data[reg_data['Sector'] == sector_name].copy()
    
    if sector_df.empty:
        print(f"\nError: no data found for the '{sector_name}' sector. Please check the sector name")
        return None
        
    print(f"\n1. Filtering to '{sector_name}': found {len(sector_df):,} observations")
    
    # 2. define the thresholds for "high" and "low" using quartiles
    high_ai_threshold = sector_df['AI_Cumulative_Score'].dropna().quantile(0.75)
    low_return_threshold = sector_df[return_col].dropna().quantile(0.25)
    
    print(f"2. Defining thresholds:")
    print(f"   - 'high AI score' is defined as > {high_ai_threshold:.2f} (top 25%)")
    print(f"   - 'low return' is defined as < {low_return_threshold:.2%} (bottom 25%)")
    
    # 3. identify the firms that meet both criteria
    interesting_cases = sector_df[
        (sector_df['AI_Cumulative_Score'] > high_ai_threshold) &
        (sector_df[return_col] < low_return_threshold)
    ]
    
    if interesting_cases.empty:
        print("\nNo specific firms were found that met both the 'high AI' and 'low return' criteria")
        return None
        
    print(f"\n3. Identifying cases: found {len(interesting_cases)} observations meeting both criteria")
    
    # 4. prepare a clean table for analysis
    display_cols = [
        'Ticker', 'Company Name', 'Year', 'AI_Cumulative_Score', return_col
    ]
    
    # find out which individual ai factor was highest for each case
    ai_factor_cols = [
        'AI_Washing', 'Disclosure_Sentiment', 'Forward_Looking',
        'Strategic_Depth', 'Talent_Investment', 'Risk_External',
        'Risk_Non_Adoption', 'Risk_Own_Adoption'
    ]
    # ensure only factors present in the data are used
    available_ai_factors = [f for f in ai_factor_cols if f in interesting_cases.columns]
    
    if available_ai_factors:
        interesting_cases['Top_AI_Factor'] = interesting_cases[available_ai_factors].idxmax(axis=1)
        display_cols.append('Top_AI_Factor')

    report_table = interesting_cases[display_cols].sort_values(by=[return_col], ascending=True)
    
    # format the return column as a percentage
    report_table[return_col] = report_table[return_col].apply(lambda x: f"{x:.2%}")

    print("\n\nTable of interesting cases")
    print("---------------------------------------------------------------------------------------------------")
    print("The following companies had AI scores in the top quartile but returns in the bottom quartile")
    print("These are excellent candidates for a qualitative case study in the thesis")
    print("---------------------------------------------------------------------------------------------------")
    print(report_table.to_string(index=False))
    
    return report_table

# make sure 'final_reg_data' is loaded and available in the notebook
if 'final_reg_data' in locals():
    interesting_cases_df = find_interesting_cases(final_reg_data)
else:
    print("\n\nError: the 'final_reg_data' dataframe was not found")
    print("Please ensure the preceding cells in the notebook have been run to create it")

In [42]:
import pandas as pd
import numpy as np

def run_sector_quadrant_analysis(reg_data, return_col='excess_return_12mo', quantile_threshold=0.25):
    """
    Performs a comprehensive quadrant analysis for every sector to find outliers based on
    AI scores and subsequent returns.

    Args:
        reg_data (pd.DataFrame): The final regression dataframe.
        return_col (str): The return column to use for the analysis.
        quantile_threshold (float): The percentage to use for defining high/low quantiles (e.g., 0.25 for quartiles).
    """
    print("🚀 SECTOR OUTLIER & QUADRANT ANALYSIS")
    print("=" * 60)
    print(f"🎯 Objective: For each sector, find firms in all four quadrants of AI Score vs. Return.")
    print(f"   (Using top/bottom {int(quantile_threshold*100)}% for high/low thresholds)")

    all_sectors = sorted([s for s in reg_data['Sector'].unique() if pd.notna(s)])
    all_outliers_list = []

    for sector in all_sectors:
        print(f"\n\nANALYZING SECTOR: {sector.upper()}")
        print("-" * 50)
        
        sector_df = reg_data[reg_data['Sector'] == sector].copy()
        if len(sector_df) < 20:
            print("Skipping due to insufficient observations.")
            continue

        # Define high/low thresholds for AI Score and Returns for the current sector
        high_ai_thresh = sector_df['AI_Cumulative_Score'].quantile(1 - quantile_threshold)
        low_ai_thresh = sector_df['AI_Cumulative_Score'].quantile(quantile_threshold)
        high_ret_thresh = sector_df[return_col].quantile(1 - quantile_threshold)
        low_ret_thresh = sector_df[return_col].quantile(quantile_threshold)

        # Identify firms in each of the four quadrants
        quadrants = {
            "High AI / High Return (Champions)": sector_df[(sector_df['AI_Cumulative_Score'] >= high_ai_thresh) & (sector_df[return_col] >= high_ret_thresh)],
            "High AI / Low Return (AI Hypers)": sector_df[(sector_df['AI_Cumulative_Score'] >= high_ai_thresh) & (sector_df[return_col] <= low_ret_thresh)],
            "Low AI / High Return (Quiet Performers)": sector_df[(sector_df['AI_Cumulative_Score'] <= low_ai_thresh) & (sector_df[return_col] >= high_ret_thresh)],
            "Low AI / Low Return (Laggards)": sector_df[(sector_df['AI_Cumulative_Score'] <= low_ai_thresh) & (sector_df[return_col] <= low_ret_thresh)],
        }
        
        # Display results and collect data for export
        for quad_name, quad_df in quadrants.items():
            print(f"\n {quad_name}: Found {len(quad_df)} firms.")
            if not quad_df.empty:
                # Prepare table for display
                display_cols = ['Ticker', 'Company Name', 'Year', 'AI_Cumulative_Score', return_col]
                report_table = quad_df[display_cols].copy()
                # Sort by return to see the most extreme examples
                sort_asc = True if 'Low Return' in quad_name else False
                report_table = report_table.sort_values(by=return_col, ascending=sort_asc)
                report_table[return_col] = report_table[return_col].apply(lambda x: f"{x:.2%}")
                
                print(report_table.head().to_string(index=False))
                if len(report_table) > 5:
                    print(f"... and {len(report_table)-5} more.")

                # Add data to our master list for final export
                for _, row in quad_df.iterrows():
                    all_outliers_list.append({
                        'Sector': sector,
                        'Quadrant': quad_name,
                        'Ticker': row['Ticker'],
                        'Company Name': row['Company Name'],
                        'Year': row['Year'],
                        'AI_Cumulative_Score': row['AI_Cumulative_Score'],
                        'Return': row[return_col]
                    })
    
    # Export the complete list of outliers to a single CSV file
    if all_outliers_list:
        outliers_df = pd.DataFrame(all_outliers_list)
        filename = "sector_quadrant_analysis_all_outliers.csv"
        outliers_df.to_csv(filename, index=False)
        print(f"\n\n\n Complete list of all outliers from all sectors exported to: {filename}")

#================================================================================
#                                 RUN THE ANALYSIS
#================================================================================
# This calls the explorer function using your 'final_reg_data' dataframe.

if 'final_reg_data' in locals():
    # You can change the quantile_threshold to find more or fewer outliers (e.g., 0.2 for top/bottom 20%)
    run_sector_quadrant_analysis(final_reg_data, quantile_threshold=0.25)
else:
    print("\n\n❌ ERROR: The 'final_reg_data' dataframe was not found.")
    print("Please ensure you have run the preceding cells in your notebook to create it.")


🚀 SECTOR OUTLIER & QUADRANT ANALYSIS
🎯 Objective: For each sector, find firms in all four quadrants of AI Score vs. Return.
   (Using top/bottom 25% for high/low thresholds)


ANALYZING SECTOR: COMMUNICATION
--------------------------------------------------

✅ High AI / High Return (Champions): Found 3 firms.
Ticker            Company Name  Year  AI_Cumulative_Score excess_return_12mo
  SIRI SIRIUS XM HOLDINGS INC.  2023            23.666667            167.17%
 FWONK      Liberty Media Corp  2023            27.000000             36.78%
   WLY JOHN WILEY & SONS, INC.  2023            24.000000             21.98%

✅ High AI / Low Return (AI Hypers): Found 3 firms.
Ticker                         Company Name  Year  AI_Cumulative_Score excess_return_12mo
   TTD                     Trade Desk, Inc.  2020            29.333333            -72.99%
  PINS                      PINTEREST, INC.  2020            30.333333            -66.75%
   CCO Clear Channel Outdoor Holdings, Inc.  2023         

In [ ]:
# ENDOGENEITY TESTING MODULE FOR AI FACTOR THESIS
# ===============================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.stattools import durbin_watson
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT
import warnings
warnings.filterwarnings('ignore')

class EndogeneityTester:
    """
    Comprehensive endogeneity testing for AI factor thesis.
    Tests for:
    1. Omitted Variable Bias (management quality proxies)
    2. Reverse Causality (lagged performance → current AI metrics)
    3. Robustness checks with additional controls
    """
    
    def __init__(self, data_path, output_path):
        self.data_path = data_path
        self.output_path = output_path
        self.df = None
        self.results = {}
        self.diagnostics = {}
        
        # Core variables from main analysis
        self.y_var = 'excess_return_12mo_annualized'
        
        # Standard controls (same as main analysis)
        self.standard_controls_config = {
            'Log Market Cap': 'calc_log_market_cap',
            'Price-to-Book': 'calc_price_to_book',
            'ROA': 'calc_roa',
            'Asset Growth': 'calc_asset_growth',
            'Momentum (12-1m)': 'calc_mom_12_1'
        }
        
        # Management quality proxies for omitted variable bias testing
        self.management_proxies_config = {
            'ROA Volatility': 'calc_roa_volatility',
            'Return Volatility': 'calc_return_volatility',
            'Earnings Quality': 'calc_earnings_quality',
            'Past Performance (t-1)': 'excess_return_12mo_annualized_lag1',
            'Past Performance (t-2)': 'excess_return_12mo_annualized_lag2',
            'Past ROA (t-1)': 'calc_roa_lag1',
            'Revenue Growth Volatility': 'calc_revenue_growth_volatility'
        }
        
        # Lagged variables for reverse causality testing
        self.lagged_performance_vars = {
            'Lagged Return (t-1)': 'excess_return_12mo_annualized_lag1',
            'Lagged Return (t-2)': 'excess_return_12mo_annualized_lag2',
            'Lagged ROA (t-1)': 'calc_roa_lag1',
            'Lagged ROA (t-2)': 'calc_roa_lag2',
            'Lagged Market Cap (t-1)': 'calc_log_market_cap_lag1',
            'Lagged Momentum (t-1)': 'calc_mom_12_1_lag1'
        }
        
        self.fe_config = {'year': True, 'sector': True}  # Year + Sector FE

    def load_and_prepare_data(self):
        """Load data and create management quality proxies"""
        print("📊 Loading data for endogeneity testing...")
        self.df = pd.read_csv(self.data_path, dtype={'gvkey': str, 'CIK': str})
        
        # Create AI factor averages (same as main analysis)
        self.ai_factors_config = {
            'Strategic Depth': ['Strategic Depth_gpt4o_Numeric', 'Strategic Depth_flash1_5_Numeric', 'Strategic Depth_flash2_5_Numeric'],
            'AI Sentiment': ['Disclosure Sentiment_gpt4o_Numeric', 'Disclosure Sentiment_flash1_5_Numeric', 'Disclosure Sentiment_flash2_5_Numeric'],
            'Risk Own Adoption': ['Risk - Own Adoption_gpt4o_Numeric', 'Risk - Own Adoption_flash1_5_Numeric', 'Risk - Own Adoption_flash2_5_Numeric'],
            'Risk External Threats': ['Risk - External Threats_gpt4o_Numeric', 'Risk - External Threats_flash1_5_Numeric', 'Risk - External Threats_flash2_5_Numeric'],
            'Risk Non-Adoption': ['Risk - Non-Adoption_gpt4o_Numeric', 'Risk - Non-Adoption_flash1_5_Numeric', 'Risk - Non-Adoption_flash2_5_Numeric'],
            'Forward Looking': ['Forward-Looking_gpt4o_Numeric', 'Forward-Looking_flash1_5_Numeric', 'Forward-Looking_flash2_5_Numeric'],
            'AI Washing': ['AI Washing Index_gpt4o_Numeric', 'AI Washing Index_flash1_5_Numeric', 'AI Washing Index_flash2_5_Numeric'],
            'Talent Investment': ['Talent & Investment_gpt4o_Numeric', 'Talent & Investment_flash1_5_Numeric', 'Talent & Investment_flash2_5_Numeric'],
        }
        
        self.average_ai_factor_cols_map = {}
        for factor_name, cols in self.ai_factors_config.items():
            available_cols = [col for col in cols if col in self.df.columns]
            if available_cols:
                avg_col_name = f"{factor_name.replace(' ', '_')}_Average"
                self.df[avg_col_name] = self.df[available_cols].mean(axis=1, skipna=True)
                self.average_ai_factor_cols_map[factor_name] = avg_col_name
                print(f"✅ Created {avg_col_name}: {self.df[avg_col_name].notna().sum():,} obs")
        
        # Create management quality proxies
        self._create_management_proxies()
        
        # Create lagged variables
        self._create_lagged_variables()
        
        # Create fixed effects dummies
        self._prepare_fixed_effects_dummies()
        
        return self

    def _create_management_proxies(self):
        """Create management quality proxy variables"""
        print("🔧 Creating management quality proxies...")
        
        # Sort by firm and year for lagging/rolling calculations
        self.df = self.df.sort_values(['gvkey', 'Year'])
        
        # ROA Volatility (3-year rolling standard deviation)
        if 'calc_roa' in self.df.columns:
            self.df['calc_roa_volatility'] = self.df.groupby('gvkey')['calc_roa'].rolling(
                window=3, min_periods=2
            ).std().reset_index(0, drop=True)
        
        # Return Volatility (past 12 months if daily returns available, or 3-year annual)
        if self.y_var in self.df.columns:
            self.df['calc_return_volatility'] = self.df.groupby('gvkey')[self.y_var].rolling(
                window=3, min_periods=2
            ).std().reset_index(0, drop=True)
        
        # Earnings Quality (persistence of ROA - correlation with lagged ROA)
        if 'calc_roa' in self.df.columns:
            self.df['calc_roa_lag1'] = self.df.groupby('gvkey')['calc_roa'].shift(1)
            # For earnings quality, we'll use the absolute difference from trend
            self.df['calc_earnings_quality'] = self.df.groupby('gvkey')['calc_roa'].rolling(
                window=3, min_periods=2
            ).apply(lambda x: -np.std(x) if len(x) > 1 else np.nan).reset_index(0, drop=True)
        
        # Revenue Growth Volatility
        if 'calc_asset_growth' in self.df.columns:  # Using asset growth as proxy if revenue growth not available
            self.df['calc_revenue_growth_volatility'] = self.df.groupby('gvkey')['calc_asset_growth'].rolling(
                window=3, min_periods=2
            ).std().reset_index(0, drop=True)
        
        print("✅ Management quality proxies created")

    def _create_lagged_variables(self):
        """Create lagged performance variables"""
        print("🔧 Creating lagged variables for reverse causality testing...")
        
        # Create lagged performance variables
        perf_vars_to_lag = [
            self.y_var, 'calc_roa', 'calc_log_market_cap', 'calc_mom_12_1'
        ]
        
        for var in perf_vars_to_lag:
            if var in self.df.columns:
                self.df[f'{var}_lag1'] = self.df.groupby('gvkey')[var].shift(1)
                self.df[f'{var}_lag2'] = self.df.groupby('gvkey')[var].shift(2)
        
        print("✅ Lagged variables created")

    def _prepare_fixed_effects_dummies(self):
        """Create fixed effects dummy variables"""
        # Year fixed effects
        self.master_year_fe_cols = []
        if 'Year' in self.df.columns:
            valid_years = sorted(self.df['Year'].dropna().unique())
            if len(valid_years) > 1:
                for year_val in valid_years[1:]:  # Drop first year as base
                    fe_col_name = f'year_{int(year_val)}'
                    self.df[fe_col_name] = (self.df['Year'] == year_val).astype(int)
                    self.master_year_fe_cols.append(fe_col_name)
        
        # Sector fixed effects
        self.master_sector_fe_cols = []
        if 'Sector' in self.df.columns:
            valid_sectors = self.df['Sector'].dropna().unique()
            if len(valid_sectors) > 1:
                for i, sector_val in enumerate(valid_sectors[1:]):  # Drop first sector as base
                    clean_sector_val = str(sector_val).replace(' ', '_').replace('&', 'and').replace('/', '_').replace('-', '_')
                    fe_col_name = f'sector_{clean_sector_val}_{i}'
                    self.df[fe_col_name] = (self.df['Sector'] == sector_val).astype(int)
                    self.master_sector_fe_cols.append(fe_col_name)

    def _run_regression_with_controls(self, y_var, x_vars, control_vars=None, include_fe=True):
        """Helper function to run regression with specified controls"""
        if control_vars is None:
            control_vars = []
        
        # Combine all variables
        all_x_vars = x_vars.copy()
        all_x_vars.extend(control_vars)
        
        if include_fe:
            all_x_vars.extend(self.master_year_fe_cols)
            all_x_vars.extend(self.master_sector_fe_cols)
        
        # Filter to available columns
        all_x_vars = [var for var in all_x_vars if var in self.df.columns]
        all_x_vars = list(set(all_x_vars))  # Remove duplicates
        
        # Prepare regression data
        required_cols = [y_var] + all_x_vars + ['gvkey']
        reg_data = self.df[required_cols].dropna()
        
        if len(reg_data) < len(all_x_vars) + 20:  # Need sufficient observations
            return None
        
        # Winsorize variables
        vars_to_winsorize = [y_var] + all_x_vars
        for var in vars_to_winsorize:
            if pd.api.types.is_numeric_dtype(reg_data[var]):
                if reg_data[var].notna().sum() > 0 and reg_data[var].nunique() > 1:
                    p1, p99 = reg_data[var].quantile([0.01, 0.99])
                    if pd.notna(p1) and pd.notna(p99) and p1 != p99:
                        reg_data[var] = reg_data[var].clip(lower=p1, upper=p99)
        
        # Run regression
        try:
            X = sm.add_constant(reg_data[all_x_vars])
            y = reg_data[y_var]
            model = OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': reg_data['gvkey']})
            return model
        except:
            return None

    def test_omitted_variable_bias(self):
        """Test for omitted variable bias using management quality proxies"""
        print("\n🔬 Testing for Omitted Variable Bias...")
        
        self.results['omitted_variable_bias'] = {}
        
        # Get available standard controls
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        
        # Get available management proxies
        mgmt_proxies = [col for col in self.management_proxies_config.values() 
                       if col and col in self.df.columns]
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            factor_results = {}
            
            # Model 1: Baseline (AI factor + standard controls + FE)
            model_baseline = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=True
            )
            if model_baseline:
                factor_results['Baseline'] = model_baseline
            
            # Model 2: Add management quality proxies
            model_with_mgmt = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls + mgmt_proxies,
                include_fe=True
            )
            if model_with_mgmt:
                factor_results['With Management Proxies'] = model_with_mgmt
            
            # Model 3: Only management proxies (to test their significance)
            model_mgmt_only = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=mgmt_proxies,
                control_vars=std_controls,
                include_fe=True
            )
            if model_mgmt_only:
                factor_results['Management Proxies Only'] = model_mgmt_only
            
            if factor_results:
                self.results['omitted_variable_bias'][ai_factor_name] = factor_results
        
        print("✅ Omitted Variable Bias testing complete")

    def test_reverse_causality(self):
        """Test for reverse causality by regressing AI metrics on lagged performance"""
        print("\n🔬 Testing for Reverse Causality...")
        
        self.results['reverse_causality'] = {}
        
        # Get available lagged performance variables
        lagged_perf_vars = [col for col in self.lagged_performance_vars.values() 
                           if col and col in self.df.columns]
        
        # Get available standard controls (for control variables in reverse regression)
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            factor_results = {}
            
            # Test 1: AI factor regressed on lagged returns (main reverse causality test)
            lagged_return_vars = [col for col in lagged_perf_vars 
                                 if 'excess_return' in col or 'calc_roa' in col]
            
            if lagged_return_vars:
                model_reverse_main = self._run_regression_with_controls(
                    y_var=ai_col,
                    x_vars=lagged_return_vars,
                    control_vars=[],  # No controls for pure reverse causality test
                    include_fe=True
                )
                if model_reverse_main:
                    factor_results['AI ~ Lagged Performance'] = model_reverse_main
            
            # Test 2: AI factor regressed on lagged returns + controls
            if lagged_return_vars:
                model_reverse_controls = self._run_regression_with_controls(
                    y_var=ai_col,
                    x_vars=lagged_return_vars,
                    control_vars=std_controls,
                    include_fe=True
                )
                if model_reverse_controls:
                    factor_results['AI ~ Lagged Performance + Controls'] = model_reverse_controls
            
            # Test 3: Forward-looking test - current performance on lagged AI
            # (Create lagged AI variables for this test)
            ai_col_lag1 = f'{ai_col}_lag1'
            if ai_col_lag1 not in self.df.columns:
                self.df[ai_col_lag1] = self.df.groupby('gvkey')[ai_col].shift(1)
            
            if ai_col_lag1 in self.df.columns:
                model_forward = self._run_regression_with_controls(
                    y_var=self.y_var,
                    x_vars=[ai_col_lag1],
                    control_vars=std_controls,
                    include_fe=True
                )
                if model_forward:
                    factor_results['Current Return ~ Lagged AI'] = model_forward
            
            if factor_results:
                self.results['reverse_causality'][ai_factor_name] = factor_results
        
        print("✅ Reverse Causality testing complete")

    def test_coefficient_stability(self):
        """Test coefficient stability across different specifications"""
        print("\n🔬 Testing Coefficient Stability...")
        
        self.results['coefficient_stability'] = {}
        
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        mgmt_proxies = [col for col in self.management_proxies_config.values() 
                       if col and col in self.df.columns]
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            factor_results = {}
            
            # Specification 1: AI factor only + FE
            model_fe_only = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=[],
                include_fe=True
            )
            if model_fe_only:
                factor_results['FE Only'] = model_fe_only
            
            # Specification 2: + Standard controls
            model_std = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=True
            )
            if model_std:
                factor_results['+ Standard Controls'] = model_std
            
            # Specification 3: + Management proxies
            model_mgmt = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls + mgmt_proxies,
                include_fe=True
            )
            if model_mgmt:
                factor_results['+ Management Proxies'] = model_mgmt
            
            # Specification 4: No fixed effects (to test FE impact)
            model_no_fe = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=False
            )
            if model_no_fe:
                factor_results['No Fixed Effects'] = model_no_fe
            
            if factor_results:
                self.results['coefficient_stability'][ai_factor_name] = factor_results
        
        print("✅ Coefficient Stability testing complete")

    def run_all_endogeneity_tests(self):
        """Run all endogeneity tests"""
        print("\n🚀 Running comprehensive endogeneity testing...")
        
        self.test_omitted_variable_bias()
        self.test_reverse_causality()
        self.test_coefficient_stability()
        
        print("✅ All endogeneity tests complete")
        return self

    def _get_significance_stars(self, pvalue):
        """Get significance stars for p-values"""
        if pd.isna(pvalue):
            return ""
        if pvalue < 0.01:
            return "***"
        elif pvalue < 0.05:
            return "**"
        elif pvalue < 0.10:
            return "*"
        else:
            return ""

    def _create_endogeneity_table(self, doc, title, results_dict, table_num):
        """Create a table for endogeneity test results"""
        doc.add_heading(f"Table {table_num}: {title}", level=2)
        
        if not results_dict:
            doc.add_paragraph("No results available for this test.")
            return
        
        # Get all AI factors and model specifications
        ai_factors = list(results_dict.keys())
        if not ai_factors:
            doc.add_paragraph("No AI factors available for this test.")
            return
        
        # Get model specifications from first AI factor
        model_specs = list(results_dict[ai_factors[0]].keys())
        
        # Create table
        table = doc.add_table(rows=1, cols=1 + len(model_specs))
        table.style = 'TableGrid'
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        
        # Header row
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "AI Factor"
        hdr_cells[0].paragraphs[0].runs[0].font.bold = True
        hdr_cells[0].paragraphs[0].runs[0].font.size = Pt(9)
        
        for i, spec in enumerate(model_specs):
            hdr_cells[i+1].text = f"({i+1})\n{spec}"
            hdr_cells[i+1].paragraphs[0].runs[0].font.bold = True
            hdr_cells[i+1].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            hdr_cells[i+1].paragraphs[0].runs[0].font.size = Pt(9)
        
        # Add rows for each AI factor
        for ai_factor in ai_factors:
            row = table.add_row().cells
            row[0].text = ai_factor
            row[0].paragraphs[0].runs[0].font.bold = True
            row[0].paragraphs[0].runs[0].font.size = Pt(9)
            
            factor_results = results_dict[ai_factor]
            for i, spec in enumerate(model_specs):
                cell = row[i+1]
                
                if spec in factor_results and factor_results[spec] is not None:
                    model = factor_results[spec]
                    
                    # For reverse causality tests, look for AI factor coefficient
                    # For other tests, look for the main variable of interest
                    if 'reverse_causality' in title.lower():
                        # In reverse causality, we're looking at lagged performance coefficients
                        # Get the first significant lagged variable
                        significant_coefs = []
                        for param_name in model.params.index:
                            if 'lag' in param_name.lower() and param_name != 'const':
                                coef = model.params[param_name]
                                pval = model.pvalues[param_name]
                                se = model.bse[param_name]
                                stars = self._get_significance_stars(pval)
                                significant_coefs.append(f"{coef:.3f}{stars}")
                        
                        if significant_coefs:
                            cell.text = significant_coefs[0]  # Show first significant coefficient
                        else:
                            cell.text = "---"
                    else:
                        # For other tests, look for the AI factor coefficient
                        ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                        if ai_col and ai_col in model.params:
                            coef = model.params[ai_col]
                            pval = model.pvalues[ai_col]
                            se = model.bse[ai_col]
                            stars = self._get_significance_stars(pval)
                            
                            # Main coefficient
                            p_coef = cell.paragraphs[0]
                            p_coef.clear()
                            run_coef = p_coef.add_run(f"{coef:.3f}{stars}")
                            run_coef.font.size = Pt(9)
                            p_coef.alignment = WD_ALIGN_PARAGRAPH.CENTER
                            
                            # Standard error
                            p_se = cell.add_paragraph()
                            run_se = p_se.add_run(f"({se:.3f})")
                            run_se.font.italic = True
                            run_se.font.size = Pt(8)
                            p_se.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        else:
                            cell.text = "---"
                else:
                    cell.text = "---"
                
                cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Add model statistics
        stats_to_show = [
            ("Observations", "nobs"),
            ("R-squared", "rsquared"),
            ("F-statistic", "fvalue")
        ]
        
        for stat_name, stat_attr in stats_to_show:
            row = table.add_row().cells
            row[0].text = stat_name
            row[0].paragraphs[0].runs[0].font.bold = True
            row[0].paragraphs[0].runs[0].font.size = Pt(9)
            
            for i, spec in enumerate(model_specs):
                cell = row[i+1]
                
                # Get first AI factor's results for this spec to show statistics
                first_ai_factor = ai_factors[0]
                if (spec in results_dict[first_ai_factor] and 
                    results_dict[first_ai_factor][spec] is not None):
                    
                    model = results_dict[first_ai_factor][spec]
                    
                    if hasattr(model, stat_attr):
                        stat_val = getattr(model, stat_attr)
                        if stat_attr == "nobs":
                            cell.text = f"{int(stat_val):,}"
                        else:
                            cell.text = f"{stat_val:.3f}"
                    else:
                        cell.text = "N/A"
                else:
                    cell.text = "---"
                
                cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        doc.add_paragraph()

    def create_endogeneity_document(self):
        """Create the endogeneity testing document"""
        print("\n📄 Creating endogeneity testing document...")
        
        doc = Document()
        
        # Set style
        style = doc.styles['Normal']
        style.font.name = 'Times New Roman'
        style.font.size = Pt(10)
        
        # Title
        title = doc.add_heading('Endogeneity Testing for AI Factor Analysis', level=0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Subtitle
        subtitle = doc.add_paragraph()
        subtitle_run = subtitle.add_run('Testing for Omitted Variable Bias and Reverse Causality')
        subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
        subtitle_run.font.size = Pt(12)
        subtitle_run.italic = True
        
        # Executive Summary
        doc.add_heading('Executive Summary', level=1)
        summary_text = """
        This document presents comprehensive endogeneity testing for the AI factor analysis. We test for two main sources of endogeneity:
        
        1. Omitted Variable Bias: We test whether the inclusion of management quality proxies (ROA volatility, past performance, earnings quality) significantly changes our AI factor coefficients.
        
        2. Reverse Causality: We test whether past performance predicts current AI disclosure patterns, which would suggest that our results are driven by firms' past success rather than genuine AI adoption effects.
        
        3. Coefficient Stability: We examine how sensitive our main results are to different model specifications and control variable sets.
        """
        doc.add_paragraph(summary_text)
        
        # Test 1: Omitted Variable Bias
        doc.add_page_break()
        doc.add_heading('Test 1: Omitted Variable Bias', level=1)
        
        doc.add_paragraph("""
        We test for omitted variable bias by including management quality proxies in our regressions. 
        If management quality is correlated with both AI disclosure patterns and future returns, 
        our baseline results may be biased. We include the following management quality proxies:
        
        • ROA Volatility (3-year rolling standard deviation)
        • Return Volatility (3-year rolling standard deviation)  
        • Earnings Quality (negative of ROA volatility)
        • Past Performance (t-1 and t-2 lagged returns)
        • Revenue Growth Volatility
        
        If coefficients remain stable after including these proxies, it suggests our results are robust to this form of omitted variable bias.
        """)
        
        if 'omitted_variable_bias' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Omitted Variable Bias Test", 
                self.results['omitted_variable_bias'], 
                "1"
            )
        
        # Test 2: Reverse Causality
        doc.add_page_break()
        doc.add_heading('Test 2: Reverse Causality', level=1)
        
        doc.add_paragraph("""
        We test for reverse causality by regressing current AI metrics on lagged performance measures. 
        If past performance strongly predicts current AI disclosure patterns, it suggests that our 
        results may be driven by successful firms being more likely to discuss AI, rather than 
        AI disclosure predicting future success.
        
        We run the following tests:
        
        • AI Factor ~ Lagged Returns (t-1, t-2) + Fixed Effects
        • AI Factor ~ Lagged Returns + Controls + Fixed Effects  
        • Current Returns ~ Lagged AI Factor + Controls (forward-looking test)
        
        Strong significance in the first two tests would indicate reverse causality concerns.
        """)
        
        if 'reverse_causality' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Reverse Causality Test", 
                self.results['reverse_causality'], 
                "2"
            )
        
        # Test 3: Coefficient Stability
        doc.add_page_break()
        doc.add_heading('Test 3: Coefficient Stability', level=1)
        
        doc.add_paragraph("""
        We examine the stability of our AI factor coefficients across different model specifications.
        Large changes in coefficients when controls are added may indicate endogeneity concerns.
        
        Model specifications:
        
        • AI Factor + Fixed Effects Only
        • AI Factor + Standard Controls + Fixed Effects
        • AI Factor + Standard Controls + Management Proxies + Fixed Effects
        • AI Factor + Standard Controls (No Fixed Effects)
        
        Stable coefficients across specifications suggest robustness to endogeneity.
        """)
        
        if 'coefficient_stability' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Coefficient Stability Test", 
                self.results['coefficient_stability'], 
                "3"
            )
        
        # Interpretation Guidelines
        doc.add_page_break()
        doc.add_heading('Interpretation Guidelines', level=1)
        
        doc.add_paragraph("""
        Interpreting Endogeneity Test Results:
        
        1. Omitted Variable Bias:
           • Small changes in AI factor coefficients when management proxies are added suggest robustness
           • Large changes (>25% coefficient change) may indicate omitted variable bias
           • Insignificant management proxies suggest they don't confound our results
        
        2. Reverse Causality:
           • Strong significance of lagged performance predicting current AI metrics indicates concern
           • Weak or insignificant relationships suggest limited reverse causality
           • Forward-looking tests (current returns ~ lagged AI) should show significance if AI truly predicts performance
        
        3. Coefficient Stability:
           • Stable coefficients across specifications indicate robustness
           • Large changes when controls are added suggest potential endogeneity
           • Comparison with/without fixed effects shows importance of unobserved heterogeneity control
        
        Overall Assessment:
        If tests show: (1) stable coefficients when management proxies added, (2) weak reverse causality, 
        and (3) stable coefficients across specifications, then endogeneity concerns are likely limited.
        """)
        
        # Detailed Results Tables in Appendix
        doc.add_page_break()
        doc.add_heading('Appendix: Detailed Regression Results', level=1)
        
        self._create_detailed_appendix_tables(doc)
        
        # General Notes
        doc.add_paragraph()
        notes_para = doc.add_paragraph()
        notes_run = notes_para.add_run(
            "Notes: All regressions use firm-clustered standard errors. Variables are winsorized at 1%/99% levels. "
            "Fixed effects include year and sector dummies. Management quality proxies include ROA volatility, "
            "return volatility, earnings quality, and lagged performance measures. "
            "*, **, *** indicate significance at 10%, 5%, and 1% levels respectively."
        )
        notes_run.font.size = Pt(8)
        notes_run.italic = True
        
        try:
            doc.save(self.output_path)
            print(f"✅ Endogeneity testing document saved to: {self.output_path}")
        except Exception as e:
            print(f"❌ Error saving document: {e}")
        
        return self

    def _create_detailed_appendix_tables(self, doc):
        """Create detailed regression tables for appendix"""
        
        # Management Quality Variables Summary
        doc.add_heading('A. Management Quality Proxy Variables', level=2)
        
        mgmt_vars_desc = {
            'ROA Volatility': 'Three-year rolling standard deviation of Return on Assets',
            'Return Volatility': 'Three-year rolling standard deviation of annual returns', 
            'Earnings Quality': 'Negative of ROA volatility (higher = more stable earnings)',
            'Past Performance (t-1)': 'One-year lagged excess returns',
            'Past Performance (t-2)': 'Two-year lagged excess returns',
            'Revenue Growth Volatility': 'Three-year rolling standard deviation of asset growth'
        }
        
        # Create description table
        table = doc.add_table(rows=1, cols=2)
        table.style = 'TableGrid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "Variable"
        hdr_cells[1].text = "Description"
        for cell in hdr_cells:
            cell.paragraphs[0].runs[0].font.bold = True
            cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        for var_name, description in mgmt_vars_desc.items():
            row = table.add_row().cells
            row[0].text = var_name
            row[1].text = description
            for cell in row:
                cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        # Summary Statistics for Management Proxies
        doc.add_paragraph()
        doc.add_heading('B. Summary Statistics for Management Quality Proxies', level=2)
        
        mgmt_proxy_cols = [col for col in self.management_proxies_config.values() 
                          if col and col in self.df.columns]
        
        if mgmt_proxy_cols:
            self._create_summary_stats_table(doc, mgmt_proxy_cols, "Management Quality Proxies")
        
        # Correlation Matrix
        doc.add_paragraph()
        doc.add_heading('C. Correlation Matrix: AI Factors vs Management Proxies', level=2)
        
        ai_cols = list(self.average_ai_factor_cols_map.values())
        all_vars_for_corr = ai_cols + mgmt_proxy_cols
        all_vars_available = [col for col in all_vars_for_corr if col in self.df.columns]
        
        if len(all_vars_available) > 1:
            self._create_correlation_table(doc, all_vars_available, "AI Factors and Management Proxies")

    def _create_summary_stats_table(self, doc, variables, title):
        """Create summary statistics table for specified variables"""
        
        summary_data = []
        for var in variables:
            if var in self.df.columns:
                data = self.df[var].dropna()
                if not data.empty:
                    var_display_name = None
                    # Find display name from config
                    for display_name, col_name in self.management_proxies_config.items():
                        if col_name == var:
                            var_display_name = display_name
                            break
                    
                    if var_display_name is None:
                        var_display_name = var
                    
                    summary_data.append({
                        'Variable': var_display_name,
                        'N': len(data),
                        'Mean': data.mean(),
                        'Std Dev': data.std(),
                        'Min': data.min(),
                        'Median': data.median(),
                        'Max': data.max()
                    })
        
        if not summary_data:
            doc.add_paragraph(f"No data available for {title} summary statistics.")
            return
        
        table = doc.add_table(rows=1, cols=7)
        table.style = 'TableGrid'
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        
        headers = ['Variable', 'N', 'Mean', 'Std Dev', 'Min', 'Median', 'Max']
        for i, header in enumerate(headers):
            table.cell(0, i).text = header
            table.cell(0, i).paragraphs[0].runs[0].font.bold = True
            table.cell(0, i).paragraphs[0].runs[0].font.size = Pt(9)
        
        for item in summary_data:
            row = table.add_row().cells
            row[0].text = item['Variable']
            row[1].text = f"{item['N']:,}"
            row[2].text = f"{item['Mean']:.3f}"
            row[3].text = f"{item['Std Dev']:.3f}"
            row[4].text = f"{item['Min']:.3f}"
            row[5].text = f"{item['Median']:.3f}"
            row[6].text = f"{item['Max']:.3f}"
            
            for cell_idx in range(len(row)):
                if cell_idx > 0:
                    row[cell_idx].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                for run in row[cell_idx].paragraphs[0].runs:
                    run.font.size = Pt(9)

    def _create_correlation_table(self, doc, variables, title):
        """Create correlation table for specified variables"""
        
        if len(variables) < 2:
            doc.add_paragraph(f"Not enough variables for {title} correlation matrix.")
            return
        
        # Get variable display names
        var_display_names = []
        for var in variables:
            display_name = var
            # Check AI factors
            for ai_name, ai_col in self.average_ai_factor_cols_map.items():
                if ai_col == var:
                    display_name = ai_name
                    break
            # Check management proxies
            for mgmt_name, mgmt_col in self.management_proxies_config.items():
                if mgmt_col == var:
                    display_name = mgmt_name
                    break
            var_display_names.append(display_name)
        
        # Calculate correlations
        vars_in_df = [var for var in variables if var in self.df.columns]
        
        if len(vars_in_df) < 2:
            doc.add_paragraph(f"Not enough variables available in data for {title} correlation matrix.")
            return
        
        corr_df = self.df[vars_in_df].corr()
        
        # Create table
        n_vars = len(vars_in_df)
        table = doc.add_table(rows=n_vars + 1, cols=n_vars + 1)
        table.style = 'TableGrid'
        
        # Header row and column
        table.cell(0, 0).text = "Variable"
        table.cell(0, 0).paragraphs[0].runs[0].font.bold = True
        table.cell(0, 0).paragraphs[0].runs[0].font.size = Pt(8)
        
        for i, var_name in enumerate(var_display_names):
            # Column headers
            table.cell(0, i + 1).text = f"({i+1})"
            table.cell(0, i + 1).paragraphs[0].runs[0].font.bold = True
            table.cell(0, i + 1).paragraphs[0].runs[0].font.size = Pt(8)
            table.cell(0, i + 1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # Row headers
            table.cell(i + 1, 0).text = f"({i+1}) {var_name}"
            table.cell(i + 1, 0).paragraphs[0].runs[0].font.bold = True
            table.cell(i + 1, 0).paragraphs[0].runs[0].font.size = Pt(8)
        
        # Fill correlation values
        for i in range(n_vars):
            for j in range(n_vars):
                corr_val = corr_df.iloc[i, j]
                table.cell(i + 1, j + 1).text = f"{corr_val:.2f}"
                table.cell(i + 1, j + 1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                table.cell(i + 1, j + 1).paragraphs[0].runs[0].font.size = Pt(8)

    def run_complete_endogeneity_analysis(self):
        """Run complete endogeneity analysis and generate document"""
        print("🚀 RUNNING COMPREHENSIVE ENDOGENEITY TESTING")
        
        try:
            self.load_and_prepare_data()
            self.run_all_endogeneity_tests()
            self.create_endogeneity_document()
            
            # Print summary results
            self._print_endogeneity_summary()
            
            return self.results
        except Exception as e:
            print(f"❌ Critical error in endogeneity testing: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def _print_endogeneity_summary(self):
        """Print summary of endogeneity test results"""
        print("\n" + "="*80)
        print("📊 ENDOGENEITY TESTING SUMMARY")
        print("="*80)
        
        # Omitted Variable Bias Summary
        if 'omitted_variable_bias' in self.results:
            print("\n1. OMITTED VARIABLE BIAS TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['omitted_variable_bias'].items():
                if 'Baseline' in results and 'With Management Proxies' in results:
                    baseline_model = results['Baseline']
                    mgmt_model = results['With Management Proxies']
                    
                    ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                    if ai_col and ai_col in baseline_model.params and ai_col in mgmt_model.params:
                        baseline_coef = baseline_model.params[ai_col]
                        mgmt_coef = mgmt_model.params[ai_col]
                        
                        if baseline_coef != 0:
                            pct_change = ((mgmt_coef - baseline_coef) / abs(baseline_coef)) * 100
                            print(f"{ai_factor}:")
                            print(f"  Baseline coef: {baseline_coef:.3f}")
                            print(f"  With mgmt proxies: {mgmt_coef:.3f}")
                            print(f"  Change: {pct_change:.1f}%")
                            
                            if abs(pct_change) < 25:
                                print(f"  Assessment: ✅ ROBUST (small change)")
                            else:
                                print(f"  Assessment: ⚠️ POTENTIAL BIAS (large change)")
        
        # Reverse Causality Summary
        if 'reverse_causality' in self.results:
            print("\n2. REVERSE CAUSALITY TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['reverse_causality'].items():
                print(f"{ai_factor}:")
                
                if 'AI ~ Lagged Performance' in results:
                    model = results['AI ~ Lagged Performance']
                    significant_lags = []
                    
                    for param_name in model.params.index:
                        if 'lag' in param_name.lower() and param_name != 'const':
                            pval = model.pvalues[param_name]
                            if pval < 0.05:
                                significant_lags.append(param_name)
                    
                    if significant_lags:
                        print(f"  ⚠️ REVERSE CAUSALITY CONCERN: {len(significant_lags)} significant lagged vars")
                    else:
                        print(f"  ✅ LIMITED REVERSE CAUSALITY: No significant lagged performance")
                
                if 'Current Return ~ Lagged AI' in results:
                    model = results['Current Return ~ Lagged AI']
                    ai_col_lag = f"{self.average_ai_factor_cols_map.get(ai_factor)}_lag1"
                    
                    if ai_col_lag in model.params:
                        pval = model.pvalues[ai_col_lag]
                        coef = model.params[ai_col_lag]
                        
                        if pval < 0.05:
                            print(f"  ✅ FORWARD PREDICTIVE POWER: Lagged AI predicts returns (coef: {coef:.3f})")
                        else:
                            print(f"  ⚠️ WEAK FORWARD PREDICTION: Lagged AI doesn't predict returns")
        
        # Coefficient Stability Summary
        if 'coefficient_stability' in self.results:
            print("\n3. COEFFICIENT STABILITY TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['coefficient_stability'].items():
                if 'FE Only' in results and '+ Management Proxies' in results:
                    fe_model = results['FE Only']
                    full_model = results['+ Management Proxies']
                    
                    ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                    if ai_col and ai_col in fe_model.params and ai_col in full_model.params:
                        fe_coef = fe_model.params[ai_col]
                        full_coef = full_model.params[ai_col]
                        
                        if fe_coef != 0:
                            pct_change = ((full_coef - fe_coef) / abs(fe_coef)) * 100
                            print(f"{ai_factor}:")
                            print(f"  FE only: {fe_coef:.3f}")
                            print(f"  Full controls: {full_coef:.3f}")
                            print(f"  Stability: {100 - abs(pct_change):.1f}%")
                            
                            if abs(pct_change) < 25:
                                print(f"  Assessment: ✅ STABLE")
                            else:
                                print(f"  Assessment: ⚠️ UNSTABLE")
        
        print("\n" + "="*80)
        print("📋 OVERALL ENDOGENEITY ASSESSMENT:")
        print("   ✅ = Low endogeneity concern")
        print("   ⚠️ = Potential endogeneity concern") 
        print("   Results should be interpreted with appropriate caution")
        print("="*80)


# Usage example and main execution
if __name__ == "__main__":
    # Set your file paths
    data_path = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings.csv"
    output_dir = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/EndogeneityTesting/"
    
    import os
    if not os.path.exists(data_path):
        print(f"❌ ERROR: Data file not found at {data_path}")
    else:
        os.makedirs(output_dir, exist_ok=True)
        output_file_path = os.path.join(output_dir, "AI_Factor_Endogeneity_Tests.docx")
        
        # Run endogeneity testing
        tester = EndogeneityTester(data_path=data_path, output_path=output_file_path)
        results = tester.run_complete_endogeneity_analysis()
        
        if results:
            print(f"\n✅ Endogeneity testing completed successfully!")
            print(f"📄 Document saved to: {output_file_path}")
        else:
            print(f"\n❌ Endogeneity testing failed.")

In [ ]:
# ROBUST MOMENTUM RETURNS CALCULATOR - CUMULATIVE APPROACH
# =========================================================

import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

class RobustMomentumCalculator:
    """
    Calculate momentum returns using cumulative daily returns approach
    This avoids price discontinuity issues from exact date matching
    """
    
    def __init__(self, daily_prices_path, final_dataset_path):
        """
        Initialize with file paths
        """
        self.daily_prices_path = daily_prices_path
        self.final_dataset_path = final_dataset_path
        
        print("🏗️ ROBUST MOMENTUM CALCULATOR INITIALIZED")
        print(f"   📈 Daily prices: {daily_prices_path}")
        print(f"   📋 Final dataset: {final_dataset_path}")
        print("   🔧 Uses cumulative daily returns to avoid price discontinuities")
    
    def load_and_examine_data(self):
        """
        Load and examine data with focus on quality
        """
        print("\n📊 LOADING AND EXAMINING DATA")
        print("=" * 45)
        
        # Load daily prices
        print("Loading daily prices...")
        daily_cols_needed = ['gvkey', 'datadate', 'prccd', 'ajexdi', 'cshoc']
        daily_sample = pd.read_csv(self.daily_prices_path, nrows=5)
        available_cols = [col for col in daily_cols_needed if col in daily_sample.columns]
        print(f"Available columns: {available_cols}")
        
        self.daily_prices = pd.read_csv(self.daily_prices_path, usecols=available_cols, dtype={'gvkey': str})
        
        # Standardize GVKEY
        self.daily_prices['gvkey'] = self.daily_prices['gvkey'].str.strip().str.zfill(6)
        self.daily_prices['datadate'] = pd.to_datetime(self.daily_prices['datadate'])
        
        # Use adjusted prices if available, otherwise raw prices
        if 'ajexdi' in self.daily_prices.columns:
            self.daily_prices['adj_price'] = self.daily_prices['prccd'] / self.daily_prices['ajexdi']
            print("   ✅ Using split/dividend adjusted prices")
        else:
            self.daily_prices['adj_price'] = self.daily_prices['prccd']
            print("   ⚠️ No adjustment factors - using raw prices")
        
        # Clean data
        original_count = len(self.daily_prices)
        self.daily_prices = self.daily_prices[
            (self.daily_prices['adj_price'].notna()) & 
            (self.daily_prices['adj_price'] > 0)
        ].copy()
        
        print(f"   🧹 Cleaned: {original_count:,} → {len(self.daily_prices):,} ({len(self.daily_prices)/original_count:.1%})")
        
        self.daily_prices = self.daily_prices.sort_values(['gvkey', 'datadate']).reset_index(drop=True)
        
        print(f"✅ Daily Prices: {self.daily_prices.shape}")
        print(f"   📅 Date range: {self.daily_prices['datadate'].min()} to {self.daily_prices['datadate'].max()}")
        print(f"   🏢 Unique GVKEYs: {self.daily_prices['gvkey'].nunique()}")
        
        # Load final dataset
        print("\nLoading final dataset...")
        self.final_dataset = pd.read_csv(self.final_dataset_path, dtype={'gvkey': str})
        self.final_dataset['gvkey'] = self.final_dataset['gvkey'].str.strip().str.zfill(6)
        self.final_dataset['filingDate'] = pd.to_datetime(self.final_dataset['filingDate'])
        
        print(f"✅ Final Dataset: {self.final_dataset.shape}")
        print(f"   🏢 Unique GVKEYs: {self.final_dataset['gvkey'].nunique()}")
        
        # Check overlap
        daily_gvkeys = set(self.daily_prices['gvkey'].unique())
        final_gvkeys = set(self.final_dataset['gvkey'].unique())
        overlap = daily_gvkeys.intersection(final_gvkeys)
        
        print(f"\n🎯 GVKEY OVERLAP: {len(overlap)}/{len(final_gvkeys)} ({len(overlap)/len(final_gvkeys):.1%})")
        
        return self
    
    def calculate_clean_daily_returns(self):
        """
        Calculate clean daily returns for all firms
        """
        print("\n📈 CALCULATING CLEAN DAILY RETURNS")
        print("=" * 45)
        
        # Calculate daily returns with robust cleaning
        self.daily_prices['price_lag'] = self.daily_prices.groupby('gvkey')['adj_price'].shift(1)
        self.daily_prices['daily_return'] = (self.daily_prices['adj_price'] / self.daily_prices['price_lag']) - 1
        
        # Remove extreme daily returns and missing values  
        original_count = len(self.daily_prices)
        self.daily_prices = self.daily_prices[
            (self.daily_prices['daily_return'].between(-0.50, 0.50)) &  # ±50% max daily (realistic for legitimate moves)
            (self.daily_prices['daily_return'].notna())
        ].copy()
        
        print(f"   🧹 After return filtering: {original_count:,} → {len(self.daily_prices):,}")
        print(f"   📊 Daily return stats:")
        returns = self.daily_prices['daily_return']
        print(f"      Mean: {returns.mean():.6f}")
        print(f"      Std:  {returns.std():.6f}")
        print(f"      Min:  {returns.min():.6f}")
        print(f"      Max:  {returns.max():.6f}")
        
        return self
    
    def calculate_momentum_via_cumulative_returns(self, months_back=[1, 2, 3]):
        """
        Calculate momentum using cumulative daily returns approach
        """
        print(f"\n🎯 CALCULATING MOMENTUM VIA CUMULATIVE RETURNS")
        print("=" * 55)
        print(f"   📅 Months back: {months_back}")
        print(f"   🔧 Method: Cumulative daily returns over ~21 trading days with winsorization")
        
        momentum_results = []
        
        # Get overlapping firms
        daily_gvkeys = set(self.daily_prices['gvkey'].unique())
        final_gvkeys = set(self.final_dataset['gvkey'].unique())
        processable_gvkeys = daily_gvkeys.intersection(final_gvkeys)
        
        print(f"   🏢 Processing {len(processable_gvkeys)} firms")
        
        # Statistics tracking
        stats = {month: {'attempts': 0, 'successful': 0, 'avg_days': 0} for month in months_back}
        
        for gvkey in tqdm(processable_gvkeys, desc="Calculating momentum"):
            # Get firm data
            firm_data = self.daily_prices[self.daily_prices['gvkey'] == gvkey].copy()
            firm_filings = self.final_dataset[self.final_dataset['gvkey'] == gvkey].copy()
            
            if len(firm_data) < 100:  # Need sufficient history
                continue
            
            # Calculate log returns for better aggregation properties
            firm_data['log_return'] = np.log(1 + firm_data['daily_return'])
            firm_data = firm_data.sort_values('datadate')
            
            # Process each filing
            for _, filing_row in firm_filings.iterrows():
                filing_date = filing_row['filingDate']
                year = filing_row.get('Year', filing_date.year)
                
                result = {
                    'gvkey': gvkey,
                    'Year': year,
                    'filing_date': filing_date
                }
                
                # Calculate returns for each lookback period
                for months in months_back:
                    stats[months]['attempts'] += 1
                    
                    # Define lookback period (approximately months * 21 trading days)
                    target_days = months * 21
                    lookback_start = filing_date - pd.DateOffset(days=int(months * 35))  # Buffer for weekends
                    lookback_end = filing_date - pd.DateOffset(days=5)  # Small buffer before filing
                    
                    # Get data in lookback window
                    window_data = firm_data[
                        (firm_data['datadate'] >= lookback_start) &
                        (firm_data['datadate'] <= lookback_end)
                    ].copy()
                    
                    if len(window_data) < 10:  # Need minimum observations
                        result[f'return_t_minus_{months}m'] = np.nan
                        result[f'n_days_t_minus_{months}m'] = 0
                        continue
                    
                    try:
                        # Take the most recent 'target_days' observations
                        recent_data = window_data.tail(min(target_days, len(window_data)))
                        
                        # Calculate cumulative return using simple compounding
                        daily_returns = recent_data['daily_return'].values
                        cumulative_return = np.prod(1 + daily_returns) - 1
                        
                        # Store results (no hard caps - will winsorize later)
                        result[f'return_t_minus_{months}m'] = cumulative_return
                        result[f'n_days_t_minus_{months}m'] = len(recent_data)
                        result[f'start_date_t_minus_{months}m'] = recent_data['datadate'].iloc[0]
                        result[f'end_date_t_minus_{months}m'] = recent_data['datadate'].iloc[-1]
                        
                        # Update statistics
                        stats[months]['successful'] += 1
                        stats[months]['avg_days'] += len(recent_data)
                        
                    except Exception as e:
                        result[f'return_t_minus_{months}m'] = np.nan
                        result[f'n_days_t_minus_{months}m'] = 0
                        continue
                
                momentum_results.append(result)
        
        # Calculate final statistics
        for month in stats:
            if stats[month]['successful'] > 0:
                stats[month]['avg_days'] /= stats[month]['successful']
        
        if momentum_results:
            self.momentum_data = pd.DataFrame(momentum_results)
            
            # Apply winsorization to handle remaining extreme values
            print(f"\n🔧 APPLYING WINSORIZATION TO MOMENTUM RETURNS:")
            for month in months_back:
                return_col = f'return_t_minus_{month}m'
                if return_col in self.momentum_data.columns:
                    original_returns = self.momentum_data[return_col].dropna()
                    if len(original_returns) > 0:
                        # Winsorize at 1st and 99th percentiles
                        p01 = original_returns.quantile(0.01)
                        p99 = original_returns.quantile(0.99)
                        
                        winsorized_count = ((original_returns < p01) | (original_returns > p99)).sum()
                        
                        # Apply winsorization
                        self.momentum_data[return_col] = self.momentum_data[return_col].clip(lower=p01, upper=p99)
                        
                        print(f"   t-{month}m: winsorized {winsorized_count} values at [{p01:.3f}, {p99:.3f}]")
            
            print(f"\n✅ Momentum calculations completed:")
            print(f"   📊 Total observations: {len(self.momentum_data)}")
            
            # Success rates
            print(f"\n📊 Success Rate by Lookback Period:")
            for month in months_back:
                total = stats[month]['attempts']
                success = stats[month]['successful']
                avg_days = stats[month]['avg_days']
                rate = success/total*100 if total > 0 else 0
                print(f"   t-{month}m: {success:4d}/{total:4d} ({rate:5.1f}%) | Avg days: {avg_days:5.1f}")
            
            # Return statistics AFTER winsorization
            print(f"\n📊 Momentum Return Statistics (After Winsorization):")
            for month in months_back:
                return_col = f'return_t_minus_{month}m'
                if return_col in self.momentum_data.columns:
                    returns = self.momentum_data[return_col].dropna()
                    if len(returns) > 0:
                        print(f"   t-{month}m: mean={returns.mean():7.4f}, std={returns.std():6.4f}, "
                              f"min={returns.min():7.4f}, max={returns.max():7.4f}")
                        
                        # Check for remaining extremes
                        extreme_count = ((returns < -0.5) | (returns > 0.5)).sum()
                        if extreme_count > 0:
                            print(f"           ⚠️ {extreme_count} returns still >50% (after winsorization)")
                        else:
                            print(f"           ✅ All returns within reasonable bounds")
        else:
            raise ValueError("No momentum returns calculated!")
        
        return self
    
    def merge_with_final_dataset(self):
        """
        Merge momentum data with final dataset
        """
        print("\n🔗 MERGING WITH FINAL DATASET")
        print("=" * 40)
        
        # Merge on gvkey and Year
        momentum_cols = [col for col in self.momentum_data.columns 
                        if col not in ['gvkey', 'Year', 'filing_date']]
        merge_cols = ['gvkey', 'Year'] + momentum_cols
        
        self.final_dataset_with_momentum = pd.merge(
            self.final_dataset,
            self.momentum_data[merge_cols],
            on=['gvkey', 'Year'],
            how='left'
        )
        
        total_obs = len(self.final_dataset)
        
        print(f"✅ Merge completed:")
        print(f"   📊 Total observations: {total_obs}")
        print(f"   📋 New momentum columns: {len(momentum_cols)}")
        
        # Availability check
        print(f"\n📊 Final momentum availability:")
        for month in [1, 2, 3]:
            return_col = f'return_t_minus_{month}m'
            if return_col in self.final_dataset_with_momentum.columns:
                available = self.final_dataset_with_momentum[return_col].notna().sum()
                print(f"   t-{month}m: {available:4d}/{total_obs:4d} ({available/total_obs:.1%})")
        
        return self
    
    def validate_momentum_quality(self):
        """
        Perform quality checks on momentum returns
        """
        print(f"\n🔍 MOMENTUM QUALITY VALIDATION")
        print("=" * 40)
        
        for month in [1, 2, 3]:
            return_col = f'return_t_minus_{month}m'
            if return_col in self.final_dataset_with_momentum.columns:
                returns = self.final_dataset_with_momentum[return_col].dropna()
                
                if len(returns) > 0:
                    print(f"\n📊 t-{month}m Quality Check:")
                    print(f"   Valid observations: {len(returns)}")
                    print(f"   Mean: {returns.mean():8.4f}")
                    print(f"   Std:  {returns.std():8.4f}")
                    print(f"   Skew: {returns.skew():8.4f}")
                    print(f"   Kurt: {returns.kurtosis():8.4f}")
                    
                    # Percentile analysis
                    percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
                    pct_values = returns.quantile([p/100 for p in percentiles])
                    print(f"   Percentiles: " + " | ".join([f"p{p}={pct_values[p/100]:.3f}" for p in [1, 5, 95, 99]]))
                    
                    # Flag suspicious patterns
                    if abs(returns.mean()) > 0.05:
                        print(f"   ⚠️ High average return ({returns.mean():.3f}) - check for data issues")
                    if returns.std() > 0.3:
                        print(f"   ⚠️ High volatility ({returns.std():.3f}) - possible remaining outliers")
                    if abs(returns.skew()) > 3:
                        print(f"   ⚠️ High skewness ({returns.skew():.2f}) - asymmetric distribution")
                    
                    # Check extreme values
                    extreme_positive = (returns > 0.5).sum()
                    extreme_negative = (returns < -0.5).sum()
                    if extreme_positive + extreme_negative > 0:
                        print(f"   ⚠️ Extreme values: {extreme_positive} >50%, {extreme_negative} <-50%")
                    else:
                        print(f"   ✅ No extreme values detected")
        
        return self
    
    def save_enhanced_dataset(self, output_path=None):
        """
        Save enhanced dataset
        """
        print("\n💾 SAVING ENHANCED DATASET")
        print("=" * 35)
        
        if output_path is None:
            output_path = self.final_dataset_path.replace('.csv', '_with_robust_momentum.csv')
        
        self.final_dataset_with_momentum.to_csv(output_path, index=False)
        
        print(f"✅ Dataset saved: {output_path}")
        print(f"   📊 Shape: {self.final_dataset_with_momentum.shape}")
        
        return output_path
    
    def run_complete_analysis(self):
        """
        Run complete robust momentum analysis
        """
        print("🚀 RUNNING ROBUST MOMENTUM ANALYSIS")
        print("=" * 45)
        
        try:
            self.load_and_examine_data()
            self.calculate_clean_daily_returns()
            self.calculate_momentum_via_cumulative_returns()
            self.merge_with_final_dataset()
            self.validate_momentum_quality()
            output_path = self.save_enhanced_dataset()
            
            print(f"\n🎉 ROBUST MOMENTUM ANALYSIS COMPLETE!")
            print(f"   ✅ Used cumulative daily returns approach")
            print(f"   ✅ Applied strict outlier filtering")
            print(f"   ✅ Quality validated")
            print(f"   🚀 Ready for regression analysis!")
            
            return self.final_dataset_with_momentum, output_path
            
        except Exception as e:
            print(f"\n❌ ERROR: {str(e)}")
            import traceback
            traceback.print_exc()
            return None, None

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("🎯 ROBUST MOMENTUM CALCULATOR")
    print("=" * 35)
    
    calculator = RobustMomentumCalculator(
        daily_prices_path="/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Controls/daily.csv",
        final_dataset_path="/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings.csv"
    )
    
    enhanced_dataset, output_path = calculator.run_complete_analysis()
    
    if enhanced_dataset is not None:
        print(f"\n✅ SUCCESS! Robust momentum returns calculated!")
        print(f"Output: {output_path}")
    else:
        print(f"\n❌ Analysis failed")

In [ ]:
# UPDATED ENDOGENEITY TESTING MODULE FOR AI FACTOR THESIS
# ===============================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.stattools import durbin_watson
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT
import warnings
warnings.filterwarnings('ignore')

class UpdatedEndogeneityTester:
    """
    Comprehensive endogeneity testing for AI factor thesis.
    Updated to work with current dataframe structure including:
    - Momentum returns (return_t_minus_1m, return_t_minus_2m, return_t_minus_3m)
    - Factor loadings (beta_mktrf_t3, beta_mktrf_t6, etc.)
    - AI factors from multiple LLMs
    - Calculated financial metrics
    """
    
    def __init__(self, data_path, output_path):
        self.data_path = data_path
        self.output_path = output_path
        self.df = None
        self.results = {}
        self.diagnostics = {}
        
        # Primary dependent variable
        self.y_var = 'excess_return_12mo_annualized'
        
        # Standard controls (updated for current dataframe)
        self.standard_controls_config = {
            'Log Market Cap': 'calc_log_market_cap',
            'Price-to-Book': 'calc_price_to_book', 
            'ROA': 'calc_roa',
            'Market Beta': 'beta_mktrf_t6',  # Use 6-month beta
            'Momentum (t-1m)': 'return_t_minus_1m'  # Use momentum returns we just calculated
        }
        
        # Management quality proxies (updated)
        self.management_proxies_config = {
            'ROA': 'calc_roa',
            'ROE': 'calc_roe', 
            'Operating Margin': 'calc_operating_margin',
            'Profit Margin': 'calc_profit_margin',
            'Debt-to-Assets': 'calc_debt_to_assets',
            'Asset Turnover': 'calc_asset_turnover',
            'Price-to-Earnings': 'calc_price_to_earnings'
        }
        
        # Fixed effects configuration
        self.fe_config = {'year': True, 'sector': True}

    def load_and_prepare_data(self):
        """Load data and create necessary variables"""
        print("📊 Loading data for endogeneity testing...")
        self.df = pd.read_csv(self.data_path, dtype={'gvkey': str, 'CIK': str})
        
        print(f"✅ Loaded dataset: {self.df.shape}")
        print(f"   Available columns: {len(self.df.columns)}")
        print(f"   Sample period: {self.df['Year'].min()}-{self.df['Year'].max()}")
        
        # Create AI factor averages (same methodology as main analysis)
        self.ai_factors_config = {
            'Strategic Depth': ['Strategic Depth_gpt4o_Numeric', 'Strategic Depth_flash1_5_Numeric', 'Strategic Depth_flash2_5_Numeric'],
            'AI Sentiment': ['Disclosure Sentiment_gpt4o_Numeric', 'Disclosure Sentiment_flash1_5_Numeric', 'Disclosure Sentiment_flash2_5_Numeric'],
            'Risk Own Adoption': ['Risk - Own Adoption_gpt4o_Numeric', 'Risk - Own Adoption_flash1_5_Numeric', 'Risk - Own Adoption_flash2_5_Numeric'],
            'Risk External Threats': ['Risk - External Threats_gpt4o_Numeric', 'Risk - External Threats_flash1_5_Numeric', 'Risk - External Threats_flash2_5_Numeric'],
            'Risk Non-Adoption': ['Risk - Non-Adoption_gpt4o_Numeric', 'Risk - Non-Adoption_flash1_5_Numeric', 'Risk - Non-Adoption_flash2_5_Numeric'],
            'Forward Looking': ['Forward-Looking_gpt4o_Numeric', 'Forward-Looking_flash1_5_Numeric', 'Forward-Looking_flash2_5_Numeric'],
            'AI Washing': ['AI Washing Index_gpt4o_Numeric', 'AI Washing Index_flash1_5_Numeric', 'AI Washing Index_flash2_5_Numeric'],
            'Talent Investment': ['Talent & Investment_gpt4o_Numeric', 'Talent & Investment_flash1_5_Numeric', 'Talent & Investment_flash2_5_Numeric']
        }
        
        self.average_ai_factor_cols_map = {}
        for factor_name, cols in self.ai_factors_config.items():
            available_cols = [col for col in cols if col in self.df.columns]
            if available_cols:
                avg_col_name = f"{factor_name.replace(' ', '_')}_Average"
                self.df[avg_col_name] = self.df[available_cols].mean(axis=1, skipna=True)
                self.average_ai_factor_cols_map[factor_name] = avg_col_name
                print(f"✅ Created {avg_col_name}: {self.df[avg_col_name].notna().sum():,} obs")
        
        # Create additional management proxies and lagged variables
        self._create_management_proxies()
        self._create_lagged_variables()
        self._prepare_fixed_effects_dummies()
        
        return self

    def _create_management_proxies(self):
        """Create additional management quality proxy variables"""
        print("🔧 Creating management quality proxies...")
        
        # Sort by firm and year for lagging/rolling calculations
        self.df = self.df.sort_values(['gvkey', 'Year'])
        
        # Create volatility measures using rolling windows
        rolling_vars = {
            'calc_roa_volatility': 'calc_roa',
            'calc_roe_volatility': 'calc_roe', 
            'calc_return_volatility': self.y_var,
            'calc_profit_margin_volatility': 'calc_profit_margin'
        }
        
        for vol_var, base_var in rolling_vars.items():
            if base_var in self.df.columns:
                self.df[vol_var] = self.df.groupby('gvkey')[base_var].rolling(
                    window=3, min_periods=2
                ).std().reset_index(0, drop=True)
        
        # Create earnings quality measure (negative of ROA volatility)
        if 'calc_roa_volatility' in self.df.columns:
            self.df['calc_earnings_quality'] = -self.df['calc_roa_volatility']
        
        # Create momentum measures from our new momentum returns
        momentum_cols = ['return_t_minus_1m', 'return_t_minus_2m', 'return_t_minus_3m']
        available_momentum = [col for col in momentum_cols if col in self.df.columns]
        
        if len(available_momentum) >= 2:
            # Create 2-month momentum average
            self.df['calc_mom_2m_avg'] = self.df[available_momentum[:2]].mean(axis=1, skipna=True)
            
        if len(available_momentum) >= 3:
            # Create 3-month momentum average  
            self.df['calc_mom_3m_avg'] = self.df[available_momentum].mean(axis=1, skipna=True)
        
        print("✅ Management quality proxies created")

    def _create_lagged_variables(self):
        """Create lagged performance variables for reverse causality testing"""
        print("🔧 Creating lagged variables for reverse causality testing...")
        
        # Variables to lag
        vars_to_lag = [
            self.y_var, 'calc_roa', 'calc_roe', 'calc_log_market_cap',
            'return_t_minus_1m', 'calc_profit_margin', 'calc_operating_margin'
        ]
        
        # Add AI factors to lagging
        ai_factor_cols = list(self.average_ai_factor_cols_map.values())
        vars_to_lag.extend(ai_factor_cols)
        
        for var in vars_to_lag:
            if var in self.df.columns:
                self.df[f'{var}_lag1'] = self.df.groupby('gvkey')[var].shift(1)
                self.df[f'{var}_lag2'] = self.df.groupby('gvkey')[var].shift(2)
        
        print("✅ Lagged variables created")

    def _prepare_fixed_effects_dummies(self):
        """Create fixed effects dummy variables"""
        print("🔧 Creating fixed effects dummies...")
        
        # Year fixed effects
        self.master_year_fe_cols = []
        if 'Year' in self.df.columns:
            valid_years = sorted(self.df['Year'].dropna().unique())
            if len(valid_years) > 1:
                for year_val in valid_years[1:]:  # Drop first year as base
                    fe_col_name = f'year_{int(year_val)}'
                    self.df[fe_col_name] = (self.df['Year'] == year_val).astype(int)
                    self.master_year_fe_cols.append(fe_col_name)
        
        # Sector fixed effects (use sector_gsector if available)
        self.master_sector_fe_cols = []
        sector_cols = ['Sector', 'sector_gsector', 'sector_ggroup']
        sector_col = None
        
        for col in sector_cols:
            if col in self.df.columns and self.df[col].notna().sum() > 0:
                sector_col = col
                break
        
        if sector_col:
            valid_sectors = self.df[sector_col].dropna().unique()
            if len(valid_sectors) > 1:
                for i, sector_val in enumerate(valid_sectors[1:]):  # Drop first as base
                    fe_col_name = f'sector_{i+1}'
                    self.df[fe_col_name] = (self.df[sector_col] == sector_val).astype(int)
                    self.master_sector_fe_cols.append(fe_col_name)
        
        print(f"✅ Created {len(self.master_year_fe_cols)} year FE, {len(self.master_sector_fe_cols)} sector FE")

    def _run_regression_with_controls(self, y_var, x_vars, control_vars=None, include_fe=True):
        """Helper function to run regression with specified controls"""
        if control_vars is None:
            control_vars = []
        
        # Combine all variables
        all_x_vars = x_vars.copy()
        all_x_vars.extend(control_vars)
        
        if include_fe:
            all_x_vars.extend(self.master_year_fe_cols)
            all_x_vars.extend(self.master_sector_fe_cols)
        
        # Filter to available columns
        all_x_vars = [var for var in all_x_vars if var in self.df.columns]
        all_x_vars = list(set(all_x_vars))  # Remove duplicates
        
        # Prepare regression data
        required_cols = [y_var] + all_x_vars + ['gvkey']
        available_required_cols = [col for col in required_cols if col in self.df.columns]
        
        reg_data = self.df[available_required_cols].dropna()
        
        if len(reg_data) < len(all_x_vars) + 20:  # Need sufficient observations
            print(f"⚠️ Insufficient data for regression: {len(reg_data)} obs, {len(all_x_vars)} vars")
            return None
        
        # Winsorize variables at 1%/99%
        vars_to_winsorize = [col for col in [y_var] + all_x_vars if col in reg_data.columns]
        for var in vars_to_winsorize:
            if pd.api.types.is_numeric_dtype(reg_data[var]):
                if reg_data[var].notna().sum() > 0 and reg_data[var].nunique() > 1:
                    p1, p99 = reg_data[var].quantile([0.01, 0.99])
                    if pd.notna(p1) and pd.notna(p99) and p1 != p99:
                        reg_data[var] = reg_data[var].clip(lower=p1, upper=p99)
        
        # Run regression with firm-clustered standard errors
        try:
            available_x_vars = [var for var in all_x_vars if var in reg_data.columns]
            if not available_x_vars:
                return None
                
            X = sm.add_constant(reg_data[available_x_vars])
            y = reg_data[y_var]
            
            # Use firm-clustered standard errors if gvkey available
            if 'gvkey' in reg_data.columns:
                model = OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': reg_data['gvkey']})
            else:
                model = OLS(y, X).fit(cov_type='HC1')  # Robust standard errors
            
            return model
        except Exception as e:
            print(f"⚠️ Regression failed: {str(e)}")
            return None

    def test_omitted_variable_bias(self):
        """Test for omitted variable bias using management quality proxies"""
        print("\n🔬 Testing for Omitted Variable Bias...")
        
        self.results['omitted_variable_bias'] = {}
        
        # Get available controls
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        
        mgmt_proxies = [col for col in self.management_proxies_config.values() 
                       if col and col in self.df.columns]
        
        print(f"   Standard controls available: {len(std_controls)}")
        print(f"   Management proxies available: {len(mgmt_proxies)}")
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            print(f"   Testing {ai_factor_name}...")
            factor_results = {}
            
            # Model 1: Baseline (AI factor + standard controls + FE)
            model_baseline = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=True
            )
            if model_baseline:
                factor_results['Baseline'] = model_baseline
            
            # Model 2: Add management quality proxies
            model_with_mgmt = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls + mgmt_proxies,
                include_fe=True
            )
            if model_with_mgmt:
                factor_results['With Management Proxies'] = model_with_mgmt
            
            # Model 3: Only management proxies (test their significance)
            model_mgmt_only = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=mgmt_proxies,
                control_vars=std_controls,
                include_fe=True
            )
            if model_mgmt_only:
                factor_results['Management Proxies Only'] = model_mgmt_only
            
            if factor_results:
                self.results['omitted_variable_bias'][ai_factor_name] = factor_results
        
        print("✅ Omitted Variable Bias testing complete")

    def test_reverse_causality(self):
        """Test for reverse causality by regressing AI metrics on lagged performance"""
        print("\n🔬 Testing for Reverse Causality...")
        
        self.results['reverse_causality'] = {}
        
        # Lagged performance variables
        lagged_performance_vars = [
            f'{self.y_var}_lag1', f'{self.y_var}_lag2',
            'calc_roa_lag1', 'calc_roa_lag2',
            'return_t_minus_1m_lag1', 'return_t_minus_2m_lag1'
        ]
        lagged_perf_available = [col for col in lagged_performance_vars 
                                if col and col in self.df.columns]
        
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        
        print(f"   Lagged performance variables available: {len(lagged_perf_available)}")
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            print(f"   Testing reverse causality for {ai_factor_name}...")
            factor_results = {}
            
            # Test 1: AI factor regressed on lagged performance (main test)
            if lagged_perf_available:
                model_reverse_main = self._run_regression_with_controls(
                    y_var=ai_col,
                    x_vars=lagged_perf_available,
                    control_vars=[],  # No controls for pure test
                    include_fe=True
                )
                if model_reverse_main:
                    factor_results['AI ~ Lagged Performance'] = model_reverse_main
            
            # Test 2: AI factor regressed on lagged performance + controls
            if lagged_perf_available:
                model_reverse_controls = self._run_regression_with_controls(
                    y_var=ai_col,
                    x_vars=lagged_perf_available,
                    control_vars=std_controls,
                    include_fe=True
                )
                if model_reverse_controls:
                    factor_results['AI ~ Lagged Performance + Controls'] = model_reverse_controls
            
            # Test 3: Forward-looking test - current performance on lagged AI
            ai_col_lag1 = f'{ai_col}_lag1'
            if ai_col_lag1 in self.df.columns:
                model_forward = self._run_regression_with_controls(
                    y_var=self.y_var,
                    x_vars=[ai_col_lag1],
                    control_vars=std_controls,
                    include_fe=True
                )
                if model_forward:
                    factor_results['Current Return ~ Lagged AI'] = model_forward
            
            if factor_results:
                self.results['reverse_causality'][ai_factor_name] = factor_results
        
        print("✅ Reverse Causality testing complete")

    def test_coefficient_stability(self):
        """Test coefficient stability across different specifications"""
        print("\n🔬 Testing Coefficient Stability...")
        
        self.results['coefficient_stability'] = {}
        
        std_controls = [col for col in self.standard_controls_config.values() 
                       if col and col in self.df.columns]
        mgmt_proxies = [col for col in self.management_proxies_config.values() 
                       if col and col in self.df.columns]
        
        for ai_factor_name, ai_col in self.average_ai_factor_cols_map.items():
            if ai_col not in self.df.columns:
                continue
                
            print(f"   Testing stability for {ai_factor_name}...")
            factor_results = {}
            
            # Specification 1: AI factor only + FE
            model_fe_only = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=[],
                include_fe=True
            )
            if model_fe_only:
                factor_results['FE Only'] = model_fe_only
            
            # Specification 2: + Standard controls
            model_std = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=True
            )
            if model_std:
                factor_results['+ Standard Controls'] = model_std
            
            # Specification 3: + Management proxies
            model_mgmt = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls + mgmt_proxies,
                include_fe=True
            )
            if model_mgmt:
                factor_results['+ Management Proxies'] = model_mgmt
            
            # Specification 4: No fixed effects (test FE impact)
            model_no_fe = self._run_regression_with_controls(
                y_var=self.y_var,
                x_vars=[ai_col],
                control_vars=std_controls,
                include_fe=False
            )
            if model_no_fe:
                factor_results['No Fixed Effects'] = model_no_fe
            
            if factor_results:
                self.results['coefficient_stability'][ai_factor_name] = factor_results
        
        print("✅ Coefficient Stability testing complete")

    def run_all_endogeneity_tests(self):
        """Run all endogeneity tests"""
        print("\n🚀 Running comprehensive endogeneity testing...")
        
        self.test_omitted_variable_bias()
        self.test_reverse_causality()
        self.test_coefficient_stability()
        
        print("✅ All endogeneity tests complete")
        return self

    def _get_significance_stars(self, pvalue):
        """Get significance stars for p-values"""
        if pd.isna(pvalue):
            return ""
        if pvalue < 0.01:
            return "***"
        elif pvalue < 0.05:
            return "**"
        elif pvalue < 0.10:
            return "*"
        else:
            return ""

    def _create_endogeneity_table(self, doc, title, results_dict, table_num):
        """Create a table for endogeneity test results"""
        doc.add_heading(f"Table {table_num}: {title}", level=2)
        
        if not results_dict:
            doc.add_paragraph("No results available for this test.")
            return
        
        # Get all AI factors and model specifications
        ai_factors = list(results_dict.keys())
        if not ai_factors:
            doc.add_paragraph("No AI factors available for this test.")
            return
        
        # Get model specifications from first AI factor
        model_specs = list(results_dict[ai_factors[0]].keys())
        
        # Create table
        table = doc.add_table(rows=1, cols=1 + len(model_specs))
        table.style = 'TableGrid'
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        
        # Header row
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "AI Factor"
        hdr_cells[0].paragraphs[0].runs[0].font.bold = True
        hdr_cells[0].paragraphs[0].runs[0].font.size = Pt(9)
        
        for i, spec in enumerate(model_specs):
            hdr_cells[i+1].text = f"({i+1})\n{spec}"
            hdr_cells[i+1].paragraphs[0].runs[0].font.bold = True
            hdr_cells[i+1].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            hdr_cells[i+1].paragraphs[0].runs[0].font.size = Pt(9)
        
        # Add rows for each AI factor
        for ai_factor in ai_factors:
            row = table.add_row().cells
            row[0].text = ai_factor
            row[0].paragraphs[0].runs[0].font.bold = True
            row[0].paragraphs[0].runs[0].font.size = Pt(9)
            
            factor_results = results_dict[ai_factor]
            for i, spec in enumerate(model_specs):
                cell = row[i+1]
                
                if spec in factor_results and factor_results[spec] is not None:
                    model = factor_results[spec]
                    
                    # For reverse causality tests, look for significant lagged variables
                    if 'reverse_causality' in title.lower():
                        significant_coefs = []
                        for param_name in model.params.index:
                            if 'lag' in param_name.lower() and param_name != 'const':
                                coef = model.params[param_name]
                                pval = model.pvalues[param_name]
                                stars = self._get_significance_stars(pval)
                                if pval < 0.10:  # Show significant coefficients
                                    significant_coefs.append(f"{coef:.3f}{stars}")
                        
                        if significant_coefs:
                            cell.text = significant_coefs[0]  # Show first significant
                        else:
                            cell.text = "None Sig."
                    else:
                        # For other tests, look for the AI factor coefficient
                        ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                        if ai_col and ai_col in model.params:
                            coef = model.params[ai_col]
                            pval = model.pvalues[ai_col]
                            se = model.bse[ai_col]
                            stars = self._get_significance_stars(pval)
                            
                            # Main coefficient
                            p_coef = cell.paragraphs[0]
                            p_coef.clear()
                            run_coef = p_coef.add_run(f"{coef:.3f}{stars}")
                            run_coef.font.size = Pt(9)
                            p_coef.alignment = WD_ALIGN_PARAGRAPH.CENTER
                            
                            # Standard error
                            p_se = cell.add_paragraph()
                            run_se = p_se.add_run(f"({se:.3f})")
                            run_se.font.italic = True
                            run_se.font.size = Pt(8)
                            p_se.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        else:
                            cell.text = "---"
                else:
                    cell.text = "---"
                
                cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Add model statistics
        stats_to_show = [
            ("Observations", "nobs"),
            ("R-squared", "rsquared"),
            ("F-statistic", "fvalue")
        ]
        
        for stat_name, stat_attr in stats_to_show:
            row = table.add_row().cells
            row[0].text = stat_name
            row[0].paragraphs[0].runs[0].font.bold = True
            row[0].paragraphs[0].runs[0].font.size = Pt(9)
            
            for i, spec in enumerate(model_specs):
                cell = row[i+1]
                
                # Get first AI factor's results for this spec to show statistics
                first_ai_factor = ai_factors[0]
                if (spec in results_dict[first_ai_factor] and 
                    results_dict[first_ai_factor][spec] is not None):
                    
                    model = results_dict[first_ai_factor][spec]
                    
                    if hasattr(model, stat_attr):
                        stat_val = getattr(model, stat_attr)
                        if stat_attr == "nobs":
                            cell.text = f"{int(stat_val):,}"
                        else:
                            cell.text = f"{stat_val:.3f}"
                    else:
                        cell.text = "N/A"
                else:
                    cell.text = "---"
                
                cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        doc.add_paragraph()

    def create_endogeneity_document(self):
        """Create the comprehensive endogeneity testing document"""
        print("\n📄 Creating endogeneity testing document...")
        
        doc = Document()
        
        # Set style
        style = doc.styles['Normal']
        style.font.name = 'Times New Roman'
        style.font.size = Pt(10)
        
        # Title
        title = doc.add_heading('Endogeneity Testing for AI Factor Analysis', level=0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Subtitle
        subtitle = doc.add_paragraph()
        subtitle_run = subtitle.add_run('Testing for Omitted Variable Bias and Reverse Causality')
        subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
        subtitle_run.font.size = Pt(12)
        subtitle_run.italic = True
        
        # Executive Summary
        doc.add_heading('Executive Summary', level=1)
        summary_text = f"""
        This document presents comprehensive endogeneity testing for the AI factor analysis using {len(self.df):,} firm-year observations 
        from {self.df['Year'].min()}-{self.df['Year'].max()}. We test for three main sources of endogeneity:
        
        1. Omitted Variable Bias: We test whether inclusion of management quality proxies (financial ratios, 
           operational metrics) significantly changes our AI factor coefficients.
        
        2. Reverse Causality: We test whether past performance predicts current AI disclosure patterns, which 
           would suggest results are driven by firms' past success rather than genuine AI adoption effects.
        
        3. Coefficient Stability: We examine sensitivity of main results to different model specifications 
           and control variable sets.
        
        Key findings are summarized in the tables below, with detailed interpretation guidelines provided.
        """
        doc.add_paragraph(summary_text)
        
        # Test 1: Omitted Variable Bias
        doc.add_page_break()
        doc.add_heading('Test 1: Omitted Variable Bias', level=1)
        
        doc.add_paragraph("""
        We test for omitted variable bias by including management quality proxies in our regressions. 
        If management quality is correlated with both AI disclosure patterns and future returns, 
        our baseline results may be biased. We include the following management quality proxies:
        
        • Financial Performance: ROA, ROE, Operating Margin, Profit Margin
        • Financial Structure: Debt-to-Assets, Asset Turnover  
        • Market Valuation: Price-to-Earnings, Price-to-Book
        • Momentum Controls: Recent stock returns (t-1, t-2, t-3 months)
        
        If coefficients remain stable after including these proxies, it suggests our results are robust 
        to this form of omitted variable bias.
        """)
        
        if 'omitted_variable_bias' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Omitted Variable Bias Test", 
                self.results['omitted_variable_bias'], 
                "1"
            )
        
        # Test 2: Reverse Causality
        doc.add_page_break()
        doc.add_heading('Test 2: Reverse Causality', level=1)
        
        doc.add_paragraph("""
        We test for reverse causality by regressing current AI metrics on lagged performance measures. 
        If past performance strongly predicts current AI disclosure patterns, it suggests that our 
        results may be driven by successful firms being more likely to discuss AI, rather than 
        AI disclosure predicting future success.
        
        We run the following tests:
        
        • AI Factor ~ Lagged Returns (t-1, t-2) + Fixed Effects
        • AI Factor ~ Lagged Returns + Controls + Fixed Effects  
        • Current Returns ~ Lagged AI Factor + Controls (forward-looking test)
        
        Strong significance in the first two tests would indicate reverse causality concerns.
        Strong significance in the third test supports genuine predictive power.
        """)
        
        if 'reverse_causality' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Reverse Causality Test", 
                self.results['reverse_causality'], 
                "2"
            )
        
        # Test 3: Coefficient Stability
        doc.add_page_break()
        doc.add_heading('Test 3: Coefficient Stability', level=1)
        
        doc.add_paragraph("""
        We examine the stability of our AI factor coefficients across different model specifications.
        Large changes in coefficients when controls are added may indicate endogeneity concerns.
        
        Model specifications:
        
        • AI Factor + Fixed Effects Only
        • AI Factor + Standard Controls + Fixed Effects
        • AI Factor + Standard Controls + Management Proxies + Fixed Effects
        • AI Factor + Standard Controls (No Fixed Effects)
        
        Stable coefficients across specifications suggest robustness to endogeneity.
        """)
        
        if 'coefficient_stability' in self.results:
            self._create_endogeneity_table(
                doc, 
                "Coefficient Stability Test", 
                self.results['coefficient_stability'], 
                "3"
            )
        
        # Interpretation Guidelines
        doc.add_page_break()
        doc.add_heading('Interpretation Guidelines', level=1)
        
        doc.add_paragraph("""
        Interpreting Endogeneity Test Results:
        
        1. Omitted Variable Bias:
           • Small changes in AI factor coefficients when management proxies are added suggest robustness
           • Large changes (>25% coefficient change) may indicate omitted variable bias
           • Insignificant management proxies suggest they don't confound our results
        
        2. Reverse Causality:
           • Strong significance of lagged performance predicting current AI metrics indicates concern
           • Weak or insignificant relationships suggest limited reverse causality
           • Forward-looking tests (current returns ~ lagged AI) should show significance if AI truly predicts performance
        
        3. Coefficient Stability:
           • Stable coefficients across specifications indicate robustness
           • Large changes when controls are added suggest potential endogeneity
           • Comparison with/without fixed effects shows importance of unobserved heterogeneity control
        
        Overall Assessment:
        If tests show: (1) stable coefficients when management proxies added, (2) weak reverse causality, 
        and (3) stable coefficients across specifications, then endogeneity concerns are likely limited.
        """)
        
        # Data Summary
        doc.add_page_break()
        doc.add_heading('Data Summary', level=1)
        
        # Create data summary table
        self._create_data_summary_table(doc)
        
        # Variable Definitions
        doc.add_heading('Variable Definitions', level=2)
        self._create_variable_definitions_table(doc)
        
        # General Notes
        doc.add_paragraph()
        notes_para = doc.add_paragraph()
        notes_run = notes_para.add_run(
            "Notes: All regressions use firm-clustered standard errors. Variables are winsorized at 1%/99% levels. "
            "Fixed effects include year and sector dummies. Management quality proxies include financial performance "
            "and operational efficiency measures. AI factors are averages across multiple LLM assessments (GPT-4o, "
            "Gemini Flash 1.5, Gemini Flash 2.5). "
            "*, **, *** indicate significance at 10%, 5%, and 1% levels respectively."
        )
        notes_run.font.size = Pt(8)
        notes_run.italic = True
        
        try:
            doc.save(self.output_path)
            print(f"✅ Endogeneity testing document saved to: {self.output_path}")
        except Exception as e:
            print(f"❌ Error saving document: {e}")
        
        return self

    def _create_data_summary_table(self, doc):
        """Create data summary table"""
        doc.add_paragraph()
        
        # Calculate summary statistics
        total_obs = len(self.df)
        total_firms = self.df['gvkey'].nunique() if 'gvkey' in self.df.columns else 'N/A'
        years = f"{self.df['Year'].min()}-{self.df['Year'].max()}" if 'Year' in self.df.columns else 'N/A'
        
        # Count available variables
        ai_factors_count = len(self.average_ai_factor_cols_map)
        std_controls_count = len([col for col in self.standard_controls_config.values() 
                                 if col and col in self.df.columns])
        mgmt_proxies_count = len([col for col in self.management_proxies_config.values() 
                                 if col and col in self.df.columns])
        
        # Create summary table
        table = doc.add_table(rows=1, cols=2)
        table.style = 'TableGrid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "Data Characteristic"
        hdr_cells[1].text = "Value"
        for cell in hdr_cells:
            cell.paragraphs[0].runs[0].font.bold = True
            cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        summary_data = [
            ("Total Observations", f"{total_obs:,}"),
            ("Unique Firms", f"{total_firms:,}" if isinstance(total_firms, int) else str(total_firms)),
            ("Time Period", years),
            ("AI Factors", str(ai_factors_count)),
            ("Standard Controls", str(std_controls_count)), 
            ("Management Proxies", str(mgmt_proxies_count)),
            ("Dependent Variable", self.y_var),
            ("Fixed Effects", "Year + Sector")
        ]
        
        for characteristic, value in summary_data:
            row = table.add_row().cells
            row[0].text = characteristic
            row[1].text = value
            for cell in row:
                cell.paragraphs[0].runs[0].font.size = Pt(9)

    def _create_variable_definitions_table(self, doc):
        """Create variable definitions table"""
        doc.add_paragraph()
        
        # Variable definitions
        variable_defs = {
            **{display_name: f"Average of {display_name} across GPT-4o, Gemini Flash 1.5, and Gemini Flash 2.5 assessments" 
               for display_name in self.average_ai_factor_cols_map.keys()},
            **{display_name: description for display_name, description in {
                'Log Market Cap': 'Natural logarithm of market capitalization',
                'Price-to-Book': 'Market value of equity divided by book value of equity',
                'ROA': 'Return on Assets (Net Income / Total Assets)',
                'Market Beta': 'Market beta from 6-month factor loading estimation',
                'Momentum (t-1m)': 'Stock return over month ending 1 month before filing date',
                'ROE': 'Return on Equity (Net Income / Shareholders Equity)',
                'Operating Margin': 'Operating Income / Revenue',
                'Profit Margin': 'Net Income / Revenue', 
                'Debt-to-Assets': 'Total Debt / Total Assets',
                'Asset Turnover': 'Revenue / Total Assets',
                'Price-to-Earnings': 'Market value per share / Earnings per share'
            }.items()}
        }
        
        table = doc.add_table(rows=1, cols=2)
        table.style = 'TableGrid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = "Variable"
        hdr_cells[1].text = "Definition"
        for cell in hdr_cells:
            cell.paragraphs[0].runs[0].font.bold = True
            cell.paragraphs[0].runs[0].font.size = Pt(9)
        
        for var_name, definition in variable_defs.items():
            row = table.add_row().cells
            row[0].text = var_name
            row[1].text = definition
            for cell in row:
                cell.paragraphs[0].runs[0].font.size = Pt(9)

    def run_complete_endogeneity_analysis(self):
        """Run complete endogeneity analysis and generate document"""
        print("🚀 RUNNING COMPREHENSIVE ENDOGENEITY TESTING")
        print("="*60)
        
        try:
            self.load_and_prepare_data()
            self.run_all_endogeneity_tests()
            self.create_endogeneity_document()
            
            # Print summary results
            self._print_endogeneity_summary()
            
            return self.results
        except Exception as e:
            print(f"❌ Critical error in endogeneity testing: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def _print_endogeneity_summary(self):
        """Print summary of endogeneity test results"""
        print("\n" + "="*80)
        print("📊 ENDOGENEITY TESTING SUMMARY")
        print("="*80)
        
        # Omitted Variable Bias Summary
        if 'omitted_variable_bias' in self.results:
            print("\n1. OMITTED VARIABLE BIAS TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['omitted_variable_bias'].items():
                if 'Baseline' in results and 'With Management Proxies' in results:
                    baseline_model = results['Baseline']
                    mgmt_model = results['With Management Proxies']
                    
                    ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                    if ai_col and ai_col in baseline_model.params and ai_col in mgmt_model.params:
                        baseline_coef = baseline_model.params[ai_col]
                        mgmt_coef = mgmt_model.params[ai_col]
                        
                        if baseline_coef != 0:
                            pct_change = ((mgmt_coef - baseline_coef) / abs(baseline_coef)) * 100
                            print(f"{ai_factor}:")
                            print(f"  Baseline coef: {baseline_coef:.3f}")
                            print(f"  With mgmt proxies: {mgmt_coef:.3f}")
                            print(f"  Change: {pct_change:.1f}%")
                            
                            if abs(pct_change) < 25:
                                print(f"  Assessment: ✅ ROBUST (small change)")
                            else:
                                print(f"  Assessment: ⚠️ POTENTIAL BIAS (large change)")
        
        # Reverse Causality Summary
        if 'reverse_causality' in self.results:
            print("\n2. REVERSE CAUSALITY TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['reverse_causality'].items():
                print(f"{ai_factor}:")
                
                if 'AI ~ Lagged Performance' in results:
                    model = results['AI ~ Lagged Performance']
                    significant_lags = []
                    
                    for param_name in model.params.index:
                        if 'lag' in param_name.lower() and param_name != 'const':
                            pval = model.pvalues[param_name]
                            if pval < 0.05:
                                significant_lags.append(param_name)
                    
                    if significant_lags:
                        print(f"  ⚠️ REVERSE CAUSALITY CONCERN: {len(significant_lags)} significant lagged vars")
                    else:
                        print(f"  ✅ LIMITED REVERSE CAUSALITY: No significant lagged performance")
                
                if 'Current Return ~ Lagged AI' in results:
                    model = results['Current Return ~ Lagged AI']
                    ai_col_lag = f"{self.average_ai_factor_cols_map.get(ai_factor)}_lag1"
                    
                    if ai_col_lag in model.params:
                        pval = model.pvalues[ai_col_lag]
                        coef = model.params[ai_col_lag]
                        
                        if pval < 0.05:
                            print(f"  ✅ FORWARD PREDICTIVE POWER: Lagged AI predicts returns (coef: {coef:.3f})")
                        else:
                            print(f"  ⚠️ WEAK FORWARD PREDICTION: Lagged AI doesn't predict returns")
        
        # Coefficient Stability Summary
        if 'coefficient_stability' in self.results:
            print("\n3. COEFFICIENT STABILITY TEST:")
            print("-" * 40)
            
            for ai_factor, results in self.results['coefficient_stability'].items():
                if 'FE Only' in results and '+ Management Proxies' in results:
                    fe_model = results['FE Only']
                    full_model = results['+ Management Proxies']
                    
                    ai_col = self.average_ai_factor_cols_map.get(ai_factor)
                    if ai_col and ai_col in fe_model.params and ai_col in full_model.params:
                        fe_coef = fe_model.params[ai_col]
                        full_coef = full_model.params[ai_col]
                        
                        if fe_coef != 0:
                            pct_change = ((full_coef - fe_coef) / abs(fe_coef)) * 100
                            print(f"{ai_factor}:")
                            print(f"  FE only: {fe_coef:.3f}")
                            print(f"  Full controls: {full_coef:.3f}")
                            print(f"  Stability: {100 - abs(pct_change):.1f}%")
                            
                            if abs(pct_change) < 25:
                                print(f"  Assessment: ✅ STABLE")
                            else:
                                print(f"  Assessment: ⚠️ UNSTABLE")
        
        print("\n" + "="*80)
        print("📋 OVERALL ENDOGENEITY ASSESSMENT:")
        print("   ✅ = Low endogeneity concern")
        print("   ⚠️ = Potential endogeneity concern") 
        print("   Results should be interpreted with appropriate caution")
        print("="*80)


# Usage example and main execution
if __name__ == "__main__":
    # Set your file paths - UPDATE THESE TO YOUR ACTUAL PATHS
    data_path = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings_with_robust_momentum.csv"
    output_dir = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/EndogeneityTesting/"
    
    import os
    if not os.path.exists(data_path):
        print(f"❌ ERROR: Data file not found at {data_path}")
        print("Please update the data_path to point to your momentum-enhanced dataset")
    else:
        os.makedirs(output_dir, exist_ok=True)
        output_file_path = os.path.join(output_dir, "AI_Factor_Endogeneity_Tests_Updated.docx")
        
        print(f"📁 Data file: {data_path}")
        print(f"📄 Output: {output_file_path}")
        
        # Run endogeneity testing
        tester = UpdatedEndogeneityTester(data_path=data_path, output_path=output_file_path)
        results = tester.run_complete_endogeneity_analysis()
        
        if results:
            print(f"\n✅ Endogeneity testing completed successfully!")
            print(f"📄 Document saved to: {output_file_path}")
            print(f"🔍 Check the console output above for summary results")
            print(f"📊 Detailed tables and analysis in the Word document")
        else:
            print(f"\n❌ Endogeneity testing failed - check error messages above")

In [ ]:
    data_path = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/Regression/final_clean_dataset_filtered_with_corrected_factor_loadings_with_robust_momentum.csv"
